<a href="https://colab.research.google.com/github/tarstars/Iccpf2025/blob/main/Untitled1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip -q install requests


In [ ]:
import json, pathlib, requests

BASE = "https://31pwr5t6ij.execute-api.eu-west-2.amazonaws.com"
TEAM = {"name":"wheezards","pl":"Python","email":"tarstars@gmail.com"}

resp = requests.post(f"{BASE}/register", json=TEAM, timeout=30)
resp.raise_for_status()
team = resp.json()  # {"id": "..."}
team_id = team["id"]
print("REGISTERED. Your SECRET id:", team_id)

# Save locally in the current runtime (ephemeral unless you copy it to Drive)
pathlib.Path("/content/icfp_id.json").write_text(json.dumps(team))
print("Saved to /content/icfp_id.json")


REGISTERED. Your SECRET id: tarstars@gmail.com 9QSCzqm4wQp0qkXXQ9nLtQ
Saved to /content/icfp_id.json


In [ ]:
import json, pathlib, requests

In [ ]:
import json, requests, pathlib

BASE = "https://31pwr5t6ij.execute-api.eu-west-2.amazonaws.com"

# Load id (or paste it directly if you prefer)
team = json.loads(pathlib.Path("/content/icfp_id.json").read_text())
team_id = team["id"]

# 1) Select toy problem
r = requests.post(f"{BASE}/select", json={"id": team_id, "problemName": "probatio"})
r.raise_for_status()
print("Selected:", r.json())

# 2) First exploration: empty route, then a couple doors
plans = ["", "0", "1", "2"]i
r = requests.post(f"{BASE}/explore", json={"id": team_id, "plans": plans})
r.raise_for_status()
data = r.json()
print("Results:", data["results"])
print("queryCount:", data["queryCount"])


Selected: {'problemName': 'probatio'}
Results: [[0], [0, 2], [0, 2], [0, 2]]
queryCount: 5


In [ ]:
o

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!cp /content/icfp_id.json /content/drive/MyDrive/icfp_id.json
print("Backed up to Drive: /content/drive/MyDrive/icfp_id.json")


MessageError: Error: credential propagation was unsuccessful

In [ ]:
# === One-shot observation + map reconstruction for "probatio" ===
import json, pathlib, requests
from itertools import product

BASE = "https://31pwr5t6ij.execute-api.eu-west-2.amazonaws.com"

# Load your secret id saved earlier (adjust path if needed)
team = json.loads(pathlib.Path("/content/icfp_id.json").read_text())
TEAM_ID = team["id"]

# Prefixes we want to probe (candidate state reps)
S = [""] + list("012345") + \
    ["".join(p) for p in product("012", repeat=2)] + \
        ["03","30"]  # a couple of cross checks

        # Suffixes to discriminate behaviours (keep short to stay cheap)
        E = ["", "0", "1", "2", "3", "4", "5"]

        # Build the batched plan set and dedup
        plans = []
        seen = set()
        for s in S:
            for e in E:
                    p = s + e
                            if p not in seen:
                                        seen.add(p)
                                                    plans.append(p)

                                                    # One /explore call to measure everything
                                                    resp = requests.post(f"{BASE}/explore", json={"id": TEAM_ID, "plans": plans}, timeout=60)
                                                    resp.raise_for_status()
                                                    data = resp.json()
                                                    results = dict(zip(plans, data["results"]))
                                                    print("queryCount after this call:", data["queryCount"])

                                                    # Build rows: Row(s)[e] = label at position |s| in the run of s+e
                                                    def row_of(s):
                                                        return tuple(results[s+e][len(s)] for e in E)

                                                        rows = {s: row_of(s) for s in S}

                                                        # Identify distinct states by unique rows (Moore machine view)
                                                        row_to_idx = {}
                                                        idx_to_rep = []
                                                        for s in sorted(S, key=len):
                                                            r = rows[s]
                                                                if r not in row_to_idx:
                                                                        row_to_idx[r] = len(idx_to_rep)
                                                                                idx_to_rep.append(s)

                                                                                print(f"Observed {len(idx_to_rep)} distinct states (expect 3 for probatio).")
                                                                                for i, rep in enumerate(idx_to_rep):
                                                                                    print(f"State {i}: rep='{rep}', label={rows[rep][0]}  row={rows[rep]}")

                                                                                    # Helper to map any prefix to a state index
                                                                                    def idx_of(prefix):
                                                                                        return row_to_idx[rows[prefix]]

                                                                                        # Build transitions delta[i][a] using representatives
                                                                                        n = len(idx_to_rep)
                                                                                        d


IndentationError: unexpected indent (ipython-input-2475718913.py, line 17)

In [ ]:
# === One-shot observation + map reconstruction for "probatio" ===
import json, pathlib, requests
from itertools import product

BASE = "https://31pwr5t6ij.execute-api.eu-west-2.amazonaws.com"

# Load your secret id saved earlier (adjust path if needed)
team = json.loads(pathlib.Path("/content/icfp_id.json").read_text())
TEAM_ID = team["id"]

# Prefixes we want to probe (candidate state reps)
S = [""] + list("012345") + \
    ["".join(p) for p in product("012", repeat=2)] + \
    ["03","30"]  # a couple of cross checks

# Suffixes to discriminate behaviours (keep short to stay cheap)
E = ["", "0", "1", "2", "3", "4", "5"]

# Build the batched plan set and dedup
plans = []
seen = set()
for s in S:
    for e in E:
        p = s + e
        if p not in seen:
            seen.add(p)
            plans.append(p)

# One /explore call to measure everything
resp = requests.post(f"{BASE}/explore", json={"id": TEAM_ID, "plans": plans}, timeout=60)
resp.raise_for_status()
data = resp.json()
results = dict(zip(plans, data["results"]))
print("queryCount after this call:", data["queryCount"])

# Build rows: Row(s)[e] = label at position |s| in the run of s+e
def row_of(s):
    return tuple(results[s+e][len(s)] for e in E)

rows = {s: row_of(s) for s in S}

# Identify distinct states by unique rows (Moore machine view)
row_to_idx = {}
idx_to_rep = []
for s in sorted(S, key=len):
    r = rows[s]
    if r not in row_to_idx:
        row_to_idx[r] = len(idx_to_rep)
        idx_to_rep.append(s)

print(f"Observed {len(idx_to_rep)} distinct states (expect 3 for probatio).")
for i, rep in enumerate(idx_to_rep):
    print(f"State {i}: rep='{rep}', label={rows[rep][0]}  row={rows[rep]}")

# Helper to map any prefix to a state index
def idx_of(prefix):
    return row_to_idx[rows[prefix]]

# Build transitions delta[i][a] using representatives
n = len(idx_to_rep)
delta = [[None]*6 for _ in range(n)]
for i, s in enumerate(idx_to_rep):
    for a in range(6):
        sp = s + str(a)
        if sp not in rows:
            rows[sp] = row_of(sp)
        delta[i][a] = idx_of(sp)

# Room labels come from E[0] == "" column
rooms = [rows[idx_to_rep[i]][0] for i in range(n)]

# Starting state is the one represented by ""
starting_idx = idx_of("")

# Build undirected connections, pairing doors on both ends
connections = []
seen_edges = set()
for i in range(n):
    for a in range(6):
        j = delta[i][a]
        for bb in range(6):
            if delta[j][bb] == i:
                key = tuple(sorted([(i,a),(j,bb)]))
                if key not in seen_edges:
                    seen_edges.add(key)
                    connections.append({"from":{"room":i,"door":a},"to":{"room":j,"door":bb}})
                break

map_obj = {"rooms": rooms, "startingRoom": starting_idx, "connections": connections}
print("\n=== Candidate map ===")
print(json.dumps(map_obj, indent=2))


queryCount after this call: 115
Observed 3 distinct states (expect 3 for probatio).
State 0: rep='', label=0  row=(0, 0, 0, 0, 0, 0, 0)
State 1: rep='0', label=2  row=(2, 2, 2, 2, 2, 2, 2)
State 2: rep='3', label=1  row=(1, 1, 1, 1, 1, 1, 1)


KeyError: '040'

In [ ]:
# === Repair & finalize "probatio" using your existing session ===
import json, pathlib, requests

BASE = "https://31pwr5t6ij.execute-api.eu-west-2.amazonaws.com"

# Load TEAM_ID if needed
try:
    TEAM_ID
except NameError:
    team = json.loads(pathlib.Path("/content/icfp_id.json").read_text())
    TEAM_ID = team["id"]

# Reuse results if you already have it; otherwise start fresh
try:
    results
except NameError:
    results = {}

# We’ll use the same discriminator set
E = ["", "0", "1", "2", "3", "4", "5"]

# Representatives we already know exist from your logs:
# State 0: rep=''  label=0
# State 1: rep='0' label=2
# State 2: rep='3' label=1
reps = ["", "0", "3"]

def fetch(plans):
    plans = [p for p in plans if p not in results]
    if not plans:
        return
    r = requests.post(f"{BASE}/explore", json={"id": TEAM_ID, "plans": plans}, timeout=60)
    r.raise_for_status()
    data = r.json()
    results.update(dict(zip(plans, data["results"])))
    print(f"Fetched {len(plans)} plans; queryCount now: {data['queryCount']}")

# Ensure we have measures for reps and all their one-step successors, for all suffixes E
needed = set()
for s in reps:
    for e in E:
        needed.add(s + e)
    for a in "012345":
        sp = s + a
        for e in E:
            needed.add(sp + e)

fetch(sorted(needed))

# Build rows for representatives (vectors over E)
def row_for(s):
    return tuple(results[s + e][len(s)] for e in E)

rows = {s: row_for(s) for s in reps}

# Index states by their row; should be exactly 3 for "probatio"
row_to_idx = {rows[reps[i]]: i for i in range(3)}

# Build transitions by matching the successor row to one of the representative rows
n = 3
delta = [[None]*6 for _ in range(n)]
for i, s in enumerate(reps):
    for a in range(6):
        sp = s + str(a)
        r = row_for(sp)
        j = row_to_idx[r]  # in probatio, successors must match one of our 3 rows
        delta[i][a] = j

# Room labels (take E[0] == "" component)
rooms = [rows[s][0] for s in reps]
starting_idx = 0  # '' is our index 0 in reps

# Build undirected connections (pair the opposite door)
connections = []
seen_edges = set()
for i


In [ ]:
# === Repair & finalize "probatio" using your existing session ===
import json, pathlib, requests

BASE = "https://31pwr5t6ij.execute-api.eu-west-2.amazonaws.com"

# Load TEAM_ID if needed
try:
    TEAM_ID
    except NameError:
        team = json.loads(pathlib.Path("/content/icfp_id.json").read_text())
            TEAM_ID = team["id"]

            # Reuse results if you already have it; otherwise start fresh
            try:
                results
                except NameError:
                    results = {}

                    # We’ll use the same discriminator set
                    E = ["", "0", "1", "2", "3", "4", "5"]

                    # Representatives we already know exist from your logs:
                    # State 0: rep=''  label=0
                    # State 1: rep='0' label=2
                    # State 2: rep='3' label=1
                    reps = ["", "0", "3"]

                    def fetch(plans):
                        plans = [p for p in plans if p not in results]
                            if not plans:
                                    return
                                        r = requests.post(f"{BASE}/explore", json={"id": TEAM_ID, "plans": plans}, timeout=60)
                                            r.raise_for_status()
                                                data = r.json()
                                                    results.update(dict(zip(plans, data["results"])))
                                                        print(f"Fetched {len(plans)} plans; queryCount now: {data['queryCount']}")

                                                        # Ensure we have measures for reps and all their one-step successors, for all suffixes E
                                                        needed = set()
                                                        for s in reps:
                                                            for e in E:
                                                                    needed.add(s + e)
                                                                        for a in "012345":
                                                                                sp = s + a
                                                                                        for e in E:
                                                                                                    needed.add(sp + e)

                                                                                                    fetch(sorted(needed))

                                                                                                    # Build rows for representatives (vectors over E)
                                                                                                    def row_for(s):
                                                                                                        return tuple(results[s + e][len(s)] for e in E)

                                                                                                        rows = {s: row_for(s) for s in reps}

                                                                                                        # Index states by their row; should be exactly 3 for "probatio"
                                                                                                        row_to_idx = {rows[reps[i]]: i for i in range(3)}

                                                                                                        # Build transitions by matching the successor row to one of the representative rows
                                                                                                        n = 3
                                                                                                        delta = [[None]*6 for _ in range(n)]
                                                                                                        for i, s in enumerate(reps):
                                                                                                            for a in range(6):
                                                                                                                    sp = s + str(a)
                                                                                                                            r = row_for(sp)
                                                                                                                                    j = row_to_idx[r]  # in probatio, successors must match one of our 3 rows
                                                                                                                                            delta[i][a] = j

                                                                                                                                            # Room labels (take E[0] == "" component)
                                                                                                                                            rooms = [rows[s][0] for s in reps]
                                                                                                                                            starting_idx = 0  # '' is our index 0 in reps

                                                                                                                                            # Build undirected connections (pair the opposite door)
                                                                                                                                            connections = []
                                                                                                                                            seen_edges = set()
                                                                                                                                            for i in range(n):
                                                                                                                                                for a in range(6):
                                                                                                                                                        j = delta[i][a]
                                                                                                                                                                # find partner door at j that returns to i
                                                                                                                                                                        b = None
                                                                                                                                                                                for bb in range(6):
                                                                                                                                                                                            if delta[j][bb] == i:
                                                                                                                                                                                                            b = bb
                                                                                                                                                                                                                            break
                                                                                                                                                                                                                                    key = tuple(sorted([(i, a), (j, b)]))
                                                                                                                                                                                                                                            if key not in seen_edges:
                                                                                                                                                                                                                                                        seen_edges.add(key)
                                                                                                                                                                                                                                                                    connections.append({
                                                                                                                                                                                                                                                                                    "from": {"room": i, "door": a},
                                                                                                                                                                                                                                                                                                    "to":   {"room": j, "door": b}
                                                                                                                                                                                                                                                                                                                })

                                                                                                                                                                                                                                                                                                                map_obj = {"rooms": rooms, "startingRoom": starting_idx, "connections": connections}
                                                                                                                                                                                                                                                                                                                print("\n=== Candidate map ===")
                                                                                                                                                                                                                                                                                                                print(json.dumps(map_obj, indent=2))

                                                                                                                                                                                                                                                                                                                # Optional submission
                                                                                                                                                                                                                                                                                                                SUBMIT = False
                                                                                                                                                                                                                                                                                                                if SUBMIT:
                                                                                                                                                                                                                                                                                                                    r = requests.post(f"{BASE}/guess", json={"id": TEAM_ID, "map": map_obj}, timeout=30)
                                                                                                                                                                                                                                                                                                                        r.raise_for_status()
                                                                                                                                                                                                                                                                                                                            print("Guess result:", r.json())


SyntaxError: invalid syntax (ipython-input-869946188.py, line 9)

In [ ]:
# === Repair & finalize "probatio" (simpler version) ===
import json, pathlib, requests

BASE = "https://31pwr5t6ij.execute-api.eu-west-2.amazonaws.com"

# Always load your team ID from the file
team = json.loads(pathlib.Path("/content/icfp_id.json").read_text())
TEAM_ID = team["id"]

# Start fresh with results (we'll fetch everything we need)
results = {}

# Discriminator set
E = ["", "0", "1", "2", "3", "4", "5"]

# Representatives you already discovered: '', '0', '3'
reps = ["", "0", "3"]

def fetch(plans):
    # Remove any plans we've already fetched
    plans = [p for p in plans if p not in results]
    if not plans:
        return
    r = requests.post(f"{BASE}/explore", json={"id": TEAM_ID, "plans": plans}, timeout=60)
    r.raise_for_status()
    data = r.json()
    results.update(dict(zip(plans, data["results"])))
    print(f"Fetched {len(plans)} plans; queryCount now:", data["queryCount"])

# Build the full set of plans to fetch: each rep + all suffixes, and each one-step successor + all suffixes
needed = set()
for s in reps:
    for e in E:
        needed.add(s + e)
    for a in "012345":
        sp = s + a
        for e in E:
            needed.add(sp + e)

# Fetch whatever is missing
fetch(sorted(needed))

# Compute rows for the reps
def row_for(s):
    return tuple(results[s + e][len(s)] for e in E)

rows = {s: row_for(s) for s in reps}

# Map each unique row to a state index
row_to_idx = {rows[reps[i]]: i for i in range(3)}

# Build transitions
n = 3
delta = [[None]*6 for _ in range(n)]
for i, s in enumerate(reps):
    for a in range(6):
        sp = s + str(a)
        r = row_for(sp)
        delta[i][a] = row_to_idx[r]

# Room labels and starting room
rooms = [rows[s][0] for s in reps]
starting_idx = 0

# Build undirected connections
connections = []
seen_edges = set()
for i in range(n):
    for a in range(6):
        j = delta[i][a]
        for bb in range(6):
            if delta[j][bb] == i:
                key = tuple(sorted([(i,a),(j,bb)]))
                if key not in seen_edges:
                    seen_edges.add(key)
                    connections.append({
                        "from":{"room":i,"door":a},
                        "to":{"room":j,"door":bb}
                    })
                break

map_obj = {"rooms": rooms, "startingRoom": starting_idx, "connections": connections}
print("\n=== Candidate map ===")
print(json.dumps(map_obj, indent=2))

# Optional submission
SUBMIT = False
if SUBMIT:
    r = requests.post(f"{BASE}/guess", json={"id": TEAM_ID, "map": map_obj}, timeout=30)
    r.raise_for_status()
    print("Guess result:", r.json())


Fetched 115 plans; queryCount now: 231

=== Candidate map ===
{
  "rooms": [
    0,
    2,
    1
  ],
  "startingRoom": 0,
  "connections": [
    {
      "from": {
        "room": 0,
        "door": 0
      },
      "to": {
        "room": 1,
        "door": 0
      }
    },
    {
      "from": {
        "room": 0,
        "door": 1
      },
      "to": {
        "room": 1,
        "door": 0
      }
    },
    {
      "from": {
        "room": 0,
        "door": 2
      },
      "to": {
        "room": 1,
        "door": 0
      }
    },
    {
      "from": {
        "room": 0,
        "door": 3
      },
      "to": {
        "room": 2,
        "door": 1
      }
    },
    {
      "from": {
        "room": 0,
        "door": 4
      },
      "to": {
        "room": 1,
        "door": 0
      }
    },
    {
      "from": {
        "room": 0,
        "door": 5
      },
      "to": {
        "room": 2,
        "door": 1
      }
    },
    {
      "from": {
        "room": 1,
        "door

In [ ]:
SUBMIT = True


In [ ]:
r = requests.post(f"{BASE}/guess", json={"id": TEAM_ID, "map": map_obj}, timeout=30)
r.raise_for_status()
print("Guess result:", r.json())


Guess result: {'correct': True}


In [ ]:
# === Compact Active Learner for the next problem (query-efficient) ===
import json, pathlib, requests, random
from collections import defaultdict

BASE = "https://31pwr5t6ij.execute-api.eu-west-2.amazonaws.com"

# 0) TEAM ID & problem
TEAM_ID = json.loads(pathlib.Path("/content/icfp_id.json").read_text())["id"]

# Pick your next problem here:
PROBLEM = "n020"   # ← change to the official next problem name you want
# If you’re unsure of names, pick one you know, or re-run with another.

# 1) Select
r = requests.post(f"{BASE}/select", json={"id": TEAM_ID, "problemName": PROBLEM}, timeout=30)
r.raise_for_status()
print("Selected:", r.json())

# 2) Learner setup
E = [""]                       # discriminators (we’ll add short ones only when needed)
S = {""}                       # prefixes (candidate state reps)
results = {}                   # plan -> label sequence cache
ROW = {}                       # prefix -> row tuple over E
alphabet = "012345"

MAX_SUFFIX_LEN = 2             # keep suffixes short (<= 2)
MAX_PREFIX_DEPTH = 4           # keep prefixes shallow to keep routes short
RANDOM_SEED = 42
random.seed(RANDOM_SEED)

def explore(plans):
    """Batch /explore; cache all results."""
    todo = [p for p in plans if p not in results]
    if not todo:
        return
    rr = requests.post(f"{BASE}/explore", json={"id": TEAM_ID, "plans": todo}, timeout=120)
    rr.raise_for_status()
    data = rr.json()
    for p, seq in zip(todo, data["results"]):
        results[p] = seq
    print(f"Fetched {len(todo)} plans; queryCount:", data["queryCount"])

def ensure_row(s):
    """Ensure we measured ROW[s] over all suffixes E."""
    plans = [s + e for e in E]
    explore(plans)
    ROW[s] = tuple(results[s+e][len(s)] for e in E)

def row_eq(s,t):
    return ROW[s] == ROW[t]

# 3) Initial measurement
ensure_row("")

# 4) Main learning loop:
#    - closure: for every s in S and each door a, ensure s+a is measured and see if it’s a new state row
#    - if ambiguous states exist (equal rows but behave differently downstream), add a short suffix to E to split them
#    - stop when no new states found and no ambiguity remains
def successors(prefix):
    if len(prefix) >= MAX_PREFIX_DEPTH:    # don’t let prefixes explode
        return []
    return [prefix + a for a in alphabet]

def discover_new_states():
    """Return number of new states added to S in this pass."""
    new_count = 0
    current_rows = {ROW[s] for s in S}
    needed_prefixes = []
    for s in list(S):
        for sp in successors(s):
            needed_prefixes.append(sp)
    # measure all sp over current E
    miss = []
    for sp in needed_prefixes:
        for e in E:
            if sp+e not in results:
                miss.append(sp+e)
    explore(miss)

    # record rows for each sp and add if new
    for sp in needed_prefixes:
        if sp not in ROW:
            ROW[sp] = tuple(results[sp+e][len(sp)] for e in E)
        if ROW[sp] not in current_rows:
            S.add(sp)         # new state discovered
            current_rows.add(ROW[sp])
            new_count += 1
    return new_count

def find_ambiguity():
    """If two different prefixes have equal rows but lead differently under some a, return (s,t). Else None."""
    # map row -> canonical rep
    row_to_rep = {}
    for s in sorted(S, key=len):
        r = ROW[s]
        if r not in row_to_rep:
            row_to_rep[r] = s
    # for each state-rep, compare 1-step successors
    reps = list(row_to_rep.values())
    for i in range(len(reps)):
        for j in range(i+1, len(reps)):
            s, t = reps[i], reps[j]
            if not row_eq(s,t):
                continue
            # same row; check successors’ rows with current E
            for a in alphabet:
                sa, ta = s+a, t+a
                # ensure rows exist for sa/ta
                for u in (sa, ta):
                    if u not in ROW:
                        ensure_row(u)
                if ROW[sa] != ROW[ta]:
                    return (s, t)  # ambiguous: equal rows but distinguishable next-step structure
    return None

def add_distinguishing_suffix(s, t):
    """Try to add a short suffix e that distinguishes s and t."""
    # Try all length-1 suffixes, then all length-2
    tried = set(E)
    candidates = [d for d in alphabet if d not in tried]
    if not candidates:
        candidates = []
    if MAX_SUFFIX_LEN >= 2:
        candidates += [a+b for a in alphabet for b in alphabet if a+b not in tried]
    for e in candidates:
        # measure s+e and t+e
        explore([s+e, t+e])
        # extend E and recompute their rows
        oldE = list(E)
        E.append(e)
        ROW[s] = tuple(results[s+ee][len(s)] for ee in E)
        ROW[t] = tuple(results[t+ee][len(t)] for ee in E)
        if ROW[s] != ROW[t]:
            print("Added distinguishing suffix:", e)
            return True
        # didn’t help—roll back suffix addition
        E.pop()
        # NOTE: rows remain consistent for oldE
    return False

rounds = 0
while True:
    rounds += 1
    new_states = discover_new_states()
    amb = find_ambiguity()
    if amb:
        s,t = amb
        if not add_distinguishing_suffix(s,t):
            # Fallback: add a random short suffix (rare)
            cand = "".join(random.choice(alphabet) for _ in range(1 if random.random()<0.7 else 2))
            print("Fallback suffix:", cand)
            E.append(cand)
            # refresh rows for all known s
            explore([s+cand for s in S])
            for s in S:
                ROW[s] = tuple(results[s+e][len(s)] for e in E)
        continue
    if new_states == 0:
        break

# 5) Build compact state set and transitions
# representatives: pick shortest prefix for each unique row
row_to_rep = {}
for s in sorted(S, key=len):
    r = ROW[s]
    if r not in row_to_rep:
        row_to_rep[r] = s
reps = list(row_to_rep.values())
reps.sort(key=len)
idx_of = {row: i for i, row in enumerate([ROW[s] for s in reps])}

# transitions via row lookup
n = len(reps)
delta = [[None]*6 for _ in range(n)]
for i, s in enumerate(reps):
    for a in range(6):
        sp = s + str(a)
        # ensure row measured for sp
        if sp not in ROW:
            ensure_row(sp)
        delta[i][a] = idx_of[ROW[sp]]

# labels from E’s empty-suffix column
# find index of "" in E (always present)
e0 = E.index("")
rooms = [ROW[s][e0] for s in reps]

# starting state index
start_idx = idx_of[ROW[""]]

# undirected connections (pair opposite doors)
connections, seen_edges = [], set()
for i in range(n):
    for a in range(6):
        j = delta[i][a]
        # find partner door at j that returns to i
        b = None
        for bb in range(6):
            if delta[j][bb] == i:
                b = bb
                break
        key = tuple(sorted([(i,a),(j,b)]))
        if key not in seen_edges:
            seen_edges.add(key)
            connections.append({
                "from": {"room": i, "door": a},
                "to":   {"room": j, "door": b}
            })

map_obj = {"rooms": rooms, "startingRoom": start_idx, "connections": connections}

print("\n=== Learned map ===")
print("States discovered:", n, " (|E| =", len(E), ")")
print(json.dumps(map_obj, indent=2))

# 6) Submit (set True when you're ready)
SUBMIT = False
if SUBMIT:
    rr = requests.post(f"{BASE}/guess", json={"id": TEAM_ID, "map": map_obj}, timeout=60)
    rr.raise_for_status()
    print("Guess result:", rr.json())


HTTPError: 400 Client Error: Bad Request for url: https://31pwr5t6ij.execute-api.eu-west-2.amazonaws.com/select

In [ ]:
import json, pathlib, requests
BASE = "https://31pwr5t6ij.execute-api.eu-west-2.amazonaws.com"
TEAM_ID = json.loads(pathlib.Path("/content/icfp_id.json").read_text())["id"]

# 1) Minimal SELECT to test the pipe (probatio always exists)
r = requests.post(f"{BASE}/select", json={"id": TEAM_ID, "problemName": "primus"}, timeout=30)
print("probatio status:", r.status_code, r.text)  # should be 200

# 2) Now try your chosen problem name; print the server's error body if it fails
PROBLEM = "primus"   # <-- replace with an exact name from the scoreboard
r = requests.post(f"{BASE}/select", json={"id": TEAM_ID, "problemName": PROBLEM}, timeout=30)
print("select status:", r.status_code)
print("response:", r.text)
r.raise_for_status()


probatio status: 200 {"problemName":"primus"}
select status: 200
response: {"problemName":"primus"}


In [ ]:
# === Compact learner for 'primus' (n≈6) ===
import json, pathlib, requests, random

BASE = "https://31pwr5t6ij.execute-api.eu-west-2.amazonaws.com"
TEAM_ID = json.loads(pathlib.Path("/content/icfp_id.json").read_text())["id"]
alphabet = "012345"

# Discriminators and prefixes (grow only as needed)
E = [""]                 # start with just epsilon
S = {""}                 # start-state representative
results = {}             # plan -> label sequence
ROW = {}                 # prefix -> row over E

MAX_PREFIX_DEPTH = 4     # keeps routes short
MAX_SUFFIX_LEN = 2
random.seed(42)

def explore(plans):
    todo = [p for p in plans if p not in results]
        if not todo: return
            r = requests.post(f"{BASE}/explore", json={"id": TEAM_ID, "plans": todo}, timeout=120)
                r.raise_for_status()
                    data = r.json()
                        for p, seq in zip(todo, data["results"]):
                                results[p] = seq
                                    print(f"Fetched {len(todo)} plans; queryCount:", data["queryCount"])

                                    def ensure_row(s):
                                        explore([s+e for e in E])
                                            ROW[s] = tuple(results[s+e][len(s)] for e in E)

                                            def successors(s):
                                                if len(s) >= MAX_PREFIX_DEPTH: return []
                                                    return [s+a for a in alphabet]

                                                    # Initial measurement
                                                    ensure_row("")

                                                    def discover_new_states():
                                                        newc = 0
                                                            current = {ROW[s] for s in S}
                                                                # ensure one-step successors measured (under current E)
                                                                    need = []
                                                                        for s in list(S):
                                                                                for sp in successors(s):
                                                                                            for e in E:
                                                                                                            if sp+e not in results:
                                                                                                                                need.append(sp+e)
                                                                                                                                    explore(need)
                                                                                                                                        # add new distinct rows as new state reps
                                                                                                                                            for s in list(S):
                                                                                                                                                    for sp in successors(s):
                                                                                                                                                                if sp not in ROW:
                                                                                                                                                                                ROW[sp] = tuple(results[sp+e][len(sp)] for e in E)
                                                                                                                                                                                            if ROW[sp] not in current:
                                                                                                                                                                                                            S.add(sp); current.add(ROW[sp]); newc += 1
                                                                                                                                                                                                                return newc

                                                                                                                                                                                                                def add_distinguishing_suffix(s, t):
                                                                                                                                                                                                                    tried = set(E)
                                                                                                                                                                                                                        cands = [d for d in alphabet if d not in tried]
                                                                                                                                                                                                                            if MAX_SUFFIX_LEN >= 2:
                                                                                                                                                                                                                                    cands += [a+b for a in alphabet for b in alphabet if a+b not in tried]
                                                                                                                                                                                                                                        for e in cands:
                                                                                                                                                                                                                                                explore([s+e, t+e])
                                                                                                                                                                                                                                                        E.append(e)
                                                                                                                                                                                                                                                                old_s, old_t = ROW[s], ROW[t]
                                                                                                                                                                                                                                                                        ROW[s] = tuple(results[s+ee][len(s)] for ee in E)
                                                                                                                                                                                                                                                                                ROW[t] = tuple(results[t+ee][len(t)] for ee in E)
                                                                                                                                                                                                                                                                                        if ROW[s] != ROW[t]:
                                                                                                                                                                                                                                                                                                    print("Added distinguishing suffix:", e)
                                                                                                                                                                                                                                                                                                                return True
                                                                                                                                                                                                                                                                                                                        E.pop()
                                                                                                                                                                                                                                                                                                                                ROW[s], ROW[t] = old_s, old_t
                                                                                                                                                                                                                                                                                                                                    return False

                                                                                                                                                                                                                                                                                                                                    def find_ambiguity():
                                                                                                                                                                                                                                                                                                                                        # find two reps with equal rows but differing 1-step structure
                                                                                                                                                                                                                                                                                                                                            row_to_rep = {}
                                                                                                                                                                                                                                                                                                                                                for s in sorted(S, key=len):
                                                                                                                                                                                                                                                                                                                                                        r = ROW[s]
                                                                                                                                                                                                                                                                                                                                                                if r not in row_to_rep:
                                                                                                                                                                                                                                                                                                                                                                            row_to_rep[r] = s
                                                                                                                                                                                                                                                                                                                                                                                reps = list(row_to_rep.values())
                                                                                                                                                                                                                                                                                                                                                                                    for i in range(len(reps)):
                                                                                                                                                                                                                                                                                                                                                                                            for j in range(i+1, len(reps)):
                                                                                                                                                                                                                                                                                                                                                                                                        s,t = reps[i], reps[j]
                                                                                                                                                                                                                                                                                                                                                                                                                    if ROW[s] != ROW[t]: continue
                                                                                                                                                                                                                                                                                                                                                                                                                                for a in alphabet:
                                                                                                                                                                                                                                                                                                                                                                                                                                                sa, ta = s+a, t+a
                                                                                                                                                                                                                                                                                                                                                                                                                                                                for u in (sa, ta):
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    if u not in ROW: ensure_row(u)
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    if ROW[sa] != ROW[ta]:
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        return s,t
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            return None

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            # Main loop
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            while True:
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                new_states = discover_new_states()
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    amb = find_ambiguity()
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        if amb:
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                s,t = amb
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        if not add_distinguishing_suffix(s,t):
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    # fallback: add a short random suffix to E and refresh
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                import random
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            e = random.choice(list(alphabet))
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        print("Fallback suffix:", e)
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    E.append(e)
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                explore([s+e for s in S])
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            for s in S:
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            ROW[s] = tuple(results[s+ee][len(s)] for ee in E)
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    continue
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        if new_states == 0:
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                break

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                # Build representatives (shortest prefix per distinct row)
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                row_to_rep = {}
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                for s in sorted(S, key=len):
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    row_to_rep.setdefault(ROW[s], s)
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    reps = [row_to_rep[r] for r in row_to_rep]
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    reps.sort(key=len)
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    idx_of_row = {ROW[s]: i for i,s in enumerate(reps)}

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    # Transitions
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    n = len(reps)
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    delta = [[None]*6 for _ in range(n)]
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    for i, s in enumerate(reps):
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        for a in range(6):
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                sp = s+str(a)
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        if sp not in ROW: ensure_row(sp)
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                delta[i][a] = idx_of_row[ROW[sp]]

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                # Labels and starting index
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                e0 = E.index("")
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                rooms = [ROW[s][e0] for s in reps]
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                start_idx = idx_of_row[ROW[""]]

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                # Undirected connections
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                connections, seen = [], set()
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                for i in range(n):
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    for a in range(6):
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            j = delta[i][a]
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    b = next(bb for bb in range(6) if delta[j][bb] == i)
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            key = tuple(sorted([(i,a),(j,b)]))
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    if key not in seen:
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                seen.add(key)
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            connections.append({"from":{"room":i,"door":a},"to":{"room":j,"door":b}})

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            map_obj = {"rooms": rooms, "startingRoom": start_idx, "connections": connections}
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            print("\n=== Learned map ===")
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            print("States discovered:", n, " |E|:", len(E))
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            import json
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            print(json.dumps(map_obj, indent=2))

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            # Submit when ready
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            SUBMIT = False
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            if SUBMIT:
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                r = requests.post(f"{BASE}/guess", json={"id": TEAM_ID, "map": map_obj}, timeout=60)
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    r.raise_for_status()
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        print("Guess result:", r.json())


IndentationError: unexpected indent (ipython-input-230231126.py, line 20)

In [ ]:
# === Compact learner for 'primus' (n≈6) ===
import json, pathlib, requests, random

BASE = "https://31pwr5t6ij.execute-api.eu-west-2.amazonaws.com"
TEAM_ID = json.loads(pathlib.Path("/content/icfp_id.json").read_text())["id"]
alphabet = "012345"

# Discriminators and prefixes (grow only as needed)
E = [""]                 # start with just epsilon
S = {""}                 # start-state representative
results = {}             # plan -> label sequence
ROW = {}                 # prefix -> row over E

MAX_PREFIX_DEPTH = 4     # keeps routes short
MAX_SUFFIX_LEN = 2
random.seed(42)

def explore(plans):
    todo = [p for p in plans if p not in results]
    if not todo: return
    r = requests.post(f"{BASE}/explore", json={"id": TEAM_ID, "plans": todo}, timeout=120)
    r.raise_for_status()
    data = r.json()
    for p, seq in zip(todo, data["results"]):
        results[p] = seq
    print(f"Fetched {len(todo)} plans; queryCount:", data["queryCount"])

def ensure_row(s):
    explore([s+e for e in E])
    ROW[s] = tuple(results[s+e][len(s)] for e in E)

def successors(s):
    if len(s) >= MAX_PREFIX_DEPTH: return []
    return [s+a for a in alphabet]

# Initial measurement
ensure_row("")

def discover_new_states():
    newc = 0
    current = {ROW[s] for s in S}
    # ensure one-step successors measured (under current E)
    need = []
    for s in list(S):
        for sp in successors(s):
            for e in E:
                if sp+e not in results:
                    need.append(sp+e)
    explore(need)
    # add new distinct rows as new state reps
    for s in list(S):
        for sp in successors(s):
            if sp not in ROW:
                ROW[sp] = tuple(results[sp+e][len(sp)] for e in E)
            if ROW[sp] not in current:
                S.add(sp); current.add(ROW[sp]); newc += 1
    return newc

def add_distinguishing_suffix(s, t):
    tried = set(E)
    cands = [d for d in alphabet if d not in tried]
    if MAX_SUFFIX_LEN >= 2:
        cands += [a+b for a in alphabet for b in alphabet if a+b not in tried]
    for e in cands:
        explore([s+e, t+e])
        E.append(e)
        old_s, old_t = ROW[s], ROW[t]
        ROW[s] = tuple(results[s+ee][len(s)] for ee in E)
        ROW[t] = tuple(results[t+ee][len(t)] for ee in E)
        if ROW[s] != ROW[t]:
            print("Added distinguishing suffix:", e)
            return True
        E.pop()
        ROW[s], ROW[t] = old_s, old_t
    return False

def find_ambiguity():
    # find two reps with equal rows but differing 1-step structure
    row_to_rep = {}
    for s in sorted(S, key=len):
        r = ROW[s]
        if r not in row_to_rep:
            row_to_rep[r] = s
    reps = list(row_to_rep.values())
    for i in range(len(reps)):
        for j in range(i+1, len(reps)):
            s,t = reps[i], reps[j]
            if ROW[s] != ROW[t]: continue
            for a in alphabet:
                sa, ta = s+a, t+a
                for u in (sa, ta):
                    if u not in ROW: ensure_row(u)
                if ROW[sa] != ROW[ta]:
                    return s,t
    return None

# Main loop
while True:
    new_states = discover_new_states()
    amb = find_ambiguity()
    if amb:
        s,t = amb
        if not add_distinguishing_suffix(s,t):
            # fallback: add a short random suffix to E and refresh
            import random
            e = random.choice(list(alphabet))
            print("Fallback suffix:", e)
            E.append(e)
            explore([s+e for s in S])
            for s in S:
                ROW[s] = tuple(results[s+ee][len(s)] for ee in E)
        continue
    if new_states == 0:
        break

# Build representatives (shortest prefix per distinct row)
row_to_rep = {}
for s in sorted(S, key=len):
    row_to_rep.setdefault(ROW[s], s)
reps = [row_to_rep[r] for r in row_to_rep]
reps.sort(key=len)
idx_of_row = {ROW[s]: i for i,s in enumerate(reps)}

# Transitions
n = len(reps)
delta = [[None]*6 for _ in range(n)]
for i, s in enumerate(reps):
    for a in range(6):
        sp = s+str(a)
        if sp not in ROW: ensure_row(sp)
        delta[i][a] = idx_of_row[ROW[sp]]

# Labels and starting index
e0 = E.index("")
rooms = [ROW[s][e0] for s in reps]
start_idx = idx_of_row[ROW[""]]

# Undirected connections
connections, seen = [], set()
for i in range(n):
    for a in range(6):
        j = delta[i][a]
        b = next(bb for bb in range(6) if delta[j][bb] == i)
        key = tuple(sorted([(i,a),(j,b)]))
        if key not in seen:
            seen.add(key)
            connections.append({"from":{"room":i,"door":a},"to":{"room":j,"door":b}})

map_obj = {"rooms": rooms, "startingRoom": start_idx, "connections": connections}
print("\n=== Learned map ===")
print("States discovered:", n, " |E|:", len(E))
import json
print(json.dumps(map_obj, indent=2))

# Submit when ready
SUBMIT = False
if SUBMIT:
    r = requests.post(f"{BASE}/guess", json={"id": TEAM_ID, "map": map_obj}, timeout=60)
    r.raise_for_status()
    print("Guess result:", r.json())


Fetched 1 plans; queryCount: 2
Fetched 6 plans; queryCount: 9
Fetched 18 plans; queryCount: 28


StopIteration: 

In [ ]:
import requests, json

BASE = "https://31pwr5t6ij.execute-api.eu-west-2.amazonaws.com"

resp = requests.get(f"{BASE}/select", timeout=30)
resp.raise_for_status()
problems = resp.json()

print("Raw response:")
print(json.dumps(problems, indent=2))

# Try to print a clean list of names if present
print("\nProblem names detected:")
names = []
for item in problems:
    if isinstance(item, str):
        names.append(item)
    elif isinstance(item, dict):
        # Common keys used by endpoints
        for key in ("name", "problemName", "id", "title"):
            if key in item:
                names.append(item[key])
                break
        else:
            names.append(str(item))
    else:
        names.append(str(item))

for i, name in enumerate(names):
    print(f"{i:2d}. {name}")


Raw response:
[
  {
    "problem": "probatio",
    "size": 3
  },
  {
    "problem": "primus",
    "size": 6
  },
  {
    "problem": "secundus",
    "size": 12
  },
  {
    "problem": "tertius",
    "size": 18
  },
  {
    "problem": "quartus",
    "size": 24
  },
  {
    "problem": "quintus",
    "size": 30
  }
]

Problem names detected:
 0. {'problem': 'probatio', 'size': 3}
 1. {'problem': 'primus', 'size': 6}
 2. {'problem': 'secundus', 'size': 12}
 3. {'problem': 'tertius', 'size': 18}
 4. {'problem': 'quartus', 'size': 24}
 5. {'problem': 'quintus', 'size': 30}


In [ ]:
# === Patch: refine E until all back-edges exist, then rebuild transitions+connections ===
import requests, json

alphabet = "012345"

def explore(plans):
    todo = [p for p in plans if p not in results]
    if not todo: return
    r = requests.post(f"{BASE}/explore", json={"id": TEAM_ID, "plans": todo}, timeout=120)
    r.raise_for_status()
    data = r.json()
    for p, seq in zip(todo, data["results"]):
        results[p] = seq
    print(f"Fetched {len(todo)} plans; queryCount:", data["queryCount"])

def ensure_row(s):
    explore([s+e for e in E])
    ROW[s] = tuple(results[s+e][len(s)] for e in E)

def refresh_all_rows():
    # measure all prefixes we already know under the (possibly larger) E
    need = []
    for s in S:
        for e in E:
            if s+e not in results:
                need.append(s+e)
    explore(need)
    for s in S:
        ROW[s] = tuple(results[s+e][len(s)] for e in E)

def rebuild_states_and_delta():
    # representatives: shortest prefix for each distinct row
    row_to_rep = {}
    for s in sorted(S, key=len):
        row_to_rep.setdefault(ROW[s], s)
    reps = [row_to_rep[r] for r in row_to_rep]
    reps.sort(key=len)

    idx_of_row = {ROW[s]: i for i, s in enumerate(reps)}
    n = len(reps)

    # transitions via row lookup
    delta = [[None]*6 for _ in range(n)]
    for i, s in enumerate(reps):
        for a in range(6):
            sp = s + str(a)
            if sp not in ROW: ensure_row(sp)
            delta[i][a] = idx_of_row[ROW[sp]]
    return reps, idx_of_row, delta

def all_back_edges_exist(delta):
    n = len(delta)
    for i in range(n):
        for a in range(6):
            j = delta[i][a]
            if not any(delta[j][bb] == i for bb in range(6)):
                return False
    return True

# 1) Rebuild once with your current E
reps, idx_of_row, delta = rebuild_states_and_delta()

# 2) If any missing back-edges, add distinguishing suffixes and retry
added = []
if not all_back_edges_exist(delta):
    # try all single-digit suffixes not in E, then length-2 if needed
    cand = [d for d in alphabet if d not in E]
    cand += [a+b for a in alphabet for b in alphabet if (a+b) not in E]
    for e in cand:
        E.append(e)
        added.append(e)
        refresh_all_rows()
        reps, idx_of_row, delta = rebuild_states_and_delta()
        if all_back_edges_exist(delta):
            print("Added distinguishing suffix(es):", added)
            break

# 3) Build final map
e0 = E.index("")
rooms = [ROW[s][e0] for s in reps]
start_idx = idx_of_row[ROW[""]]

connections, seen = [], set()
for i in range(len(reps)):
    for a in range(6):
        j = delta[i][a]
        # guaranteed to exist now
        b = next(bb for bb in range(6) if delta[j][bb] == i)
        key = tuple(sorted([(i,a),(j,b)]))
        if key not in seen:
            seen.add(key)
            connections.append({"from":{"room":i,"door":a},"to":{"room":j,"door":b}})

map_obj = {"rooms": rooms, "startingRoom": start_idx, "connections": connections}
print("\n=== Refined map ===")
print("States discovered:", len(reps), "|E|:", len(E), "Added suffixes:", added)
print(json.dumps(map_obj, indent=2))

# Uncomment to submit
# SUBMIT = True
# if SUBMIT:
#     r = requests.post(f"{BASE}/guess", json={"id": TEAM_ID, "map": map_obj}, timeout=60)
#     r.raise_for_status()
#     print("Guess result:", r.json())


KeyError: (3,)

In [ ]:
# === Robust refine+rebuild: closure under successors + add suffixes until all back-edges exist ===
import requests, json

alphabet = "012345"

def explore(plans):
    todo = [p for p in plans if p not in results]
    if not todo: return
    r = requests.post(f"{BASE}/explore", json={"id": TEAM_ID, "plans": todo}, timeout=120)
    r.raise_for_status()
    data = r.json()
    for p, seq in zip(todo, data["results"]):
        results[p] = seq
    print(f"Fetched {len(todo)} plans; queryCount:", data["queryCount"])

def ensure_row(s):
    # ensure ROW[s] over current E
    missing = [s+e for e in E if s+e not in results]
    explore(missing)
    ROW[s] = tuple(results[s+e][len(s)] for e in E)

def refresh_all_rows_for(prefixes):
    need = []
    for s in prefixes:
        for e in E:
            if s+e not in results:
                need.append(s+e)
    explore(need)
    for s in prefixes:
        ROW[s] = tuple(results[s+e][len(s)] for e in E)

def build_closed_representatives():
    """
    Return (reps, idx_of_row) where reps is closed under 1-step successors:
    every row seen at s or s+a has a representative in reps.
    """
    # Start from current S (known prefixes)
    work = set(S)
    # Ensure rows exist for all in work
    refresh_all_rows_for(work)

    # Map row -> representative (pick shortest prefix)
    row_to_rep = {}
    for s in sorted(work, key=len):
        row_to_rep.setdefault(ROW[s], s)

    # Closure under successors: if any s+a yields a row with no rep, add that sp as a rep
    changed = True
    while changed:
        changed = False
        reps_now = list(row_to_rep.values())
        # ensure rows for all successors of current reps
        succs = set()
        for s in reps_now:
            for a in alphabet:
                succs.add(s + a)
        refresh_all_rows_for(succs)

        for s in reps_now:
            for a in alphabet:
                sp = s + a
                r = ROW[sp]
                if r not in row_to_rep:
                    row_to_rep[r] = sp  # adopt sp as representative for this new row
                    changed = True

    reps = list(row_to_rep.values())
    reps.sort(key=len)
    idx_of_row = {ROW[s]: i for i, s in enumerate(reps)}
    return reps, idx_of_row

def build_delta(reps, idx_of_row):
    n = len(reps)
    delta = [[None]*6 for _ in range(n)]
    for i, s in enumerate(reps):
        for a in range(6):
            sp = s + str(a)
            if sp not in ROW: ensure_row(sp)
            delta[i][a] = idx_of_row[ROW[sp]]
    return delta

def all_back_edges_exist(delta):
    n = len(delta)
    for i in range(n):
        for a in range(6):
            j = delta[i][a]
            if not any(delta[j][bb] == i for bb in range(6)):
                return False
    return True

# Try to close & rebuild; if back-edges missing, add short suffixes and retry
added_suffixes = []
while True:
    reps, idx_of_row = build_closed_representatives()
    delta = build_delta(reps, idx_of_row)
    if all_back_edges_exist(delta):
        break
    # add a distinguishing suffix and refresh
    # try all single digits not yet in E, then length-2
    cands = [d for d in alphabet if d not in E] + [a+b for a in alphabet for b in alphabet if (a+b) not in E]
    if not cands:
        raise RuntimeError("Ran out of candidate suffixes to distinguish states.")
    e = cands[0]
    E.append(e)
    added_suffixes.append(e)
    # refresh rows for all known prefixes and their 1-step successors
    refresh_all_rows_for(S | {s+a for s in S for a in alphabet})

# Build final map
e0 = E.index("")
rooms = [ROW[s][e0] for s in reps]
start_idx = idx_of_row[ROW[""]]

connections, seen = [], set()
for i in range(len(reps)):
    for a in range(6):
        j = delta[i][a]
        b = next(bb for bb in range(6) if delta[j][bb] == i)  # now guaranteed
        key = tuple(sorted([(i,a),(j,b)]))
        if key not in seen:
            seen.add(key)
            connections.append({"from":{"room":i,"door":a},"to":{"room":j,"door":b}})

map_obj = {"rooms": rooms, "startingRoom": start_idx, "connections": connections}
print("\n=== Refined map ===")
print("States discovered:", len(reps), "|E|:", len(E), "Added suffixes:", added_suffixes)
print(json.dumps(map_obj, indent=2))

# Optional submit
# SUBMIT = True
# if SUBMIT:
#     rr = requests.post(f"{BASE}/guess", json={"id": TEAM_ID, "map": map_obj}, timeout=60)
#     rr.raise_for_status()
#     print("Guess result:", rr.json())


Fetched 21 plans; queryCount: 50
Fetched 21 plans; queryCount: 72
Fetched 21 plans; queryCount: 94
Fetched 21 plans; queryCount: 116
Fetched 21 plans; queryCount: 138
Fetched 21 plans; queryCount: 160
Fetched 21 plans; queryCount: 182
Fetched 21 plans; queryCount: 204
Fetched 21 plans; queryCount: 226
Fetched 21 plans; queryCount: 248
Fetched 21 plans; queryCount: 270
Fetched 21 plans; queryCount: 292
Fetched 21 plans; queryCount: 314
Fetched 21 plans; queryCount: 336
Fetched 21 plans; queryCount: 358
Fetched 21 plans; queryCount: 380
Fetched 21 plans; queryCount: 402
Fetched 21 plans; queryCount: 424
Fetched 21 plans; queryCount: 446
Fetched 21 plans; queryCount: 468
Fetched 21 plans; queryCount: 490
Fetched 21 plans; queryCount: 512
Fetched 21 plans; queryCount: 534
Fetched 21 plans; queryCount: 556
Fetched 21 plans; queryCount: 578
Fetched 21 plans; queryCount: 600
Fetched 21 plans; queryCount: 622
Fetched 21 plans; queryCount: 644
Fetched 21 plans; queryCount: 666
Fetched 21 plans;

RuntimeError: Ran out of candidate suffixes to distinguish states.

In [ ]:
# Reselect primus to start fresh
r = requests.post(f"{BASE}/select", json={"id": TEAM_ID, "problemName": "primus"}, timeout=30)
print("select:", r.status_code, r.text)
r.raise_for_status()


select: 400 {"error":"Error: couldn't find matching record"}


HTTPError: 400 Client Error: Bad Request for url: https://31pwr5t6ij.execute-api.eu-west-2.amazonaws.com/select

In [ ]:
import json, pathlib, requests
from collections import deque

BASE = "https://31pwr5t6ij.execute-api.eu-west-2.amazonaws.com"
TEAM_ID = json.loads(pathlib.Path("/content/icfp_id.json").read_text())["id"]
alphabet = "012345"

# --- helpers ---
results = {}            # plan -> label sequence
def explore(plans):
    todo = [p for p in plans if p not in results]
    if not todo: return
    r = requests.post(f"{BASE}/explore", json={"id": TEAM_ID, "plans": todo}, timeout=120)
    r.raise_for_status()
    data = r.json()
    for p, seq in zip(todo, data["results"]):
        results[p] = seq
    print(f"Fetched {len(todo)} plans; queryCount:", data["queryCount"])

def label_at(plan, pos):  # pos is index into the returned path
    return results[plan][pos]

# --- learner state ---
E = [""]                 # discriminators, start minimal
S = set([""])            # discovered prefixes (candidates)
ROW = {}                 # ROW[s] = tuple(results[s+e][len(s)] for e in E)
MAX_PREFIX_DEPTH = 6     # allow deeper prefixes to reveal structure

def ensure_row(s):
    # measure s+e for current E
    explore([s+e for e in E if s+e not in results])
    ROW[s] = tuple(label_at(s+e, len(s)) for e in E)

def successors(s):
    return [s+a for a in alphabet] if len(s) < MAX_PREFIX_DEPTH else []

# 1) seed with start
ensure_row("")

# 2) BFS expand prefixes; add a prefix only if it creates a NEW row
def discover_prefixes():
    q = deque([""])
    seen = set([""])
    rows_seen = {ROW[""]}
    while q:
        s = q.popleft()
        for sp in successors(s):
            if sp in seen: continue
            seen.add(sp)
            # measure sp over E
            ensure_row(sp)
            if ROW[sp] not in rows_seen:
                S.add(sp)
                rows_seen.add(ROW[sp])
            q.append(sp)

discover_prefixes()

# 3) Build representatives (shortest prefix per distinct row)
def representatives():
    row_to_rep = {}
    for s in sorted(S, key=len):
        row_to_rep.setdefault(ROW[s], s)
    reps = [row_to_rep[r] for r in row_to_rep]
    reps.sort(key=len)
    idx_of_row = {ROW[s]: i for i, s in enumerate(reps)}
    return reps, idx_of_row

reps, idx_of_row = representatives()

def build_delta(reps, idx_of_row):
    n = len(reps)
    delta = [[None]*6 for _ in range(n)]
    for i, s in enumerate(reps):
        for a in range(6):
            sp = s+str(a)
            # ensure measured
            if sp not in ROW: ensure_row(sp)
            delta[i][a] = idx_of_row[ROW[sp]]
    return delta

def all_back_edges_exist(delta):
    n = len(delta)
    for i in range(n):
        for a in range(6):
            j = delta[i][a]
            if not any(delta[j][bb] == i for bb in range(6)):
                return False
    return True

# 4) If ambiguous: find two reps with equal rows but different successor rows, add distinguishing suffix
def find_ambiguity():
    # reps with same row under current E
    row_to_list = {}
    for s in reps:
        row_to_list.setdefault(ROW[s], []).append(s)
    for row_val, group in row_to_list.items():
        if len(group) < 2: continue
        # compare successors
        for i in range(len(group)):
            for j in range(i+1, len(group)):
                s, t = group[i], group[j]
                for a in alphabet:
                    sa, ta = s+a, t+a
                    if sa not in ROW: ensure_row(sa)
                    if ta not in ROW: ensure_row(ta)
                    if ROW[sa] != ROW[ta]:
                        return s, t, a  # suffix 'a' will distinguish
    return None

# 5) Refine until back-edges exist (or no ambiguity)
while True:
    delta = build_delta(reps, idx_of_row)
    if all_back_edges_exist(delta):
        break
    amb = find_ambiguity()
    if not amb:
        # as a fallback, try length-2 distinguishing via two-step where successors mismatch
        # search pairs a,b where s+ab and t+ab rows differ
        done = False
        for a in alphabet:
            for b in alphabet:
                diffs = False
                for s in reps:
                    ensure_row(s+a+b)
                # recompute ambiguity search using 'ab'
                # measure only; deciding globally is heavy—just add the first candidate
                E.append(a+b)
                # refresh ROW for all prefixes under new E
                for s in list(S):
                    explore([s+(a+b)])
                    ROW[s] = tuple(label_at(s+e, len(s)) for e in E)
                reps, idx_of_row = representatives()
                done = True
                break
            if done: break
        if done: continue
        else:
            raise RuntimeError("Could not find distinguishing suffix — consider increasing MAX_PREFIX_DEPTH.")
    s, t, a = amb
    # add the exact distinguishing suffix
    if a not in E:
        E.append(a)
        # refresh all rows under new E
        for s0 in list(S):
            explore([s0+a])
            ROW[s0] = tuple(label_at(s0+e, len(s0)) for e in E)
        # also re-run BFS once to pick up any new rows that become visible
        discover_prefixes()
        reps, idx_of_row = representatives()
        # continue loop, rebuild delta

# 6) Build final map
e0 = E.index("")
rooms = [ROW[s][e0] for s in reps]
start_idx = idx_of_row[ROW[""]]

connections, seen = [], set()
for i in range(len(reps)):
    for a in range(6):
        j = delta[i][a]
        b = next(bb for bb in range(6) if delta[j][bb] == i)
        key = tuple(sorted([(i,a),(j,b)]))
        if key not in seen:
            seen.add(key)
            connections.append({"from":{"room":i,"door":a},"to":{"room":j,"door":b}})

print("\n=== Learned map (primus) ===")
print("States discovered:", len(reps), "|E|:", len(E))
print(json.dumps({"rooms": rooms, "startingRoom": start_idx, "connections": connections}, indent=2))

# Optionally submit:
# r = requests.post(f"{BASE}/guess", json={"id": TEAM_ID, "map": {"rooms": rooms, "startingRoom": start_idx, "connections": connections}}, timeout=60)
# print("Guess:", r.status_code, r.text)
app

Fetched 1 plans; queryCount: 2
Fetched 1 plans; queryCount: 4
Fetched 1 plans; queryCount: 6
Fetched 1 plans; queryCount: 8
Fetched 1 plans; queryCount: 10
Fetched 1 plans; queryCount: 12
Fetched 1 plans; queryCount: 14
Fetched 1 plans; queryCount: 16
Fetched 1 plans; queryCount: 18
Fetched 1 plans; queryCount: 20
Fetched 1 plans; queryCount: 22
Fetched 1 plans; queryCount: 24
Fetched 1 plans; queryCount: 26
Fetched 1 plans; queryCount: 28
Fetched 1 plans; queryCount: 30
Fetched 1 plans; queryCount: 32
Fetched 1 plans; queryCount: 34
Fetched 1 plans; queryCount: 36
Fetched 1 plans; queryCount: 38
Fetched 1 plans; queryCount: 40
Fetched 1 plans; queryCount: 42
Fetched 1 plans; queryCount: 44
Fetched 1 plans; queryCount: 46
Fetched 1 plans; queryCount: 48
Fetched 1 plans; queryCount: 50
Fetched 1 plans; queryCount: 52
Fetched 1 plans; queryCount: 54
Fetched 1 plans; queryCount: 56
Fetched 1 plans; queryCount: 58
Fetched 1 plans; queryCount: 60
Fetched 1 plans; queryCount: 62
Fetched 1 pl

KeyboardInterrupt: 

In [ ]:
# === Lean learner for primus (6 rooms) — batched + no tree explosion ===
import json, pathlib, requests

BASE = "https://31pwr5t6ij.execute-api.eu-west-2.amazonaws.com"
TEAM_ID = json.loads(pathlib.Path("/content/icfp_id.json").read_text())["id"]
alphabet = "012345"

# ---- Helpers (batched explore) ----
results = {}  # plan -> label sequence

def explore(plans, chunk=200):
    # batch missing plans; print server's queryCount after each POST
    todo = [p for p in plans if p not in results]
    if not todo: return
    for i in range(0, len(todo), chunk):
        batch = todo[i:i+chunk]
        r = requests.post(f"{BASE}/explore", json={"id": TEAM_ID, "plans": batch}, timeout=120)
        r.raise_for_status()
        data = r.json()
        for p, seq in zip(batch, data["results"]):
            results[p] = seq
        print(f"Fetched {len(batch)} plans; queryCount:", data["queryCount"])

def label_at(plan, pos):
    return results[plan][pos]

# Discriminators (keep minimal)
E = [""]

def row_of(prefix):
    # with E=[""], the row is just the label at the position |prefix|
    return (label_at(prefix, len(prefix)),)

# ---- Core idea: expand ONLY representatives and their 1-step successors ----
# Start with the empty prefix as the only representative
reps =


SyntaxError: invalid syntax (ipython-input-436890943.py, line 36)

In [ ]:
import json, pathlib, requests

BASE = "https://31pwr5t6ij.execute-api.eu-west-2.amazonaws.com"

r = requests.post(f"{BASE}/select", json={"id": TEAM_ID, "problemName": "primus"}, timeout=30)
print("select:", r.status_code, r.text)
r.raise_for_status()

select: 200 {"problemName":"primus"}


In [ ]:
BASE = "https://31pwr5t6ij.execute-api.eu-west-2.amazonaws.com"

In [ ]:
TEAM_ID = "9QSCzqm4wPQ0qkXXQ9nLtQ"

In [ ]:
import requests, random, string

BASE = "https://31pwr5t6ij.execute-api.eu-west-2.amazonaws.com"

# If you already know an ID, put it here; otherwise leave it empty:
TEAM_ID = "9QSCzqm4wPQ0qkXXQ9nLtQ".strip()

TEAM_NAME = "wheezards"
LANGUAGE  = "Python"
EMAIL     = "tarstars@gmail.com"

def try_select(team_id, problem):
    return requests.post(f"{BASE}/select", json={"id": team_id, "problemName": problem}, timeout=30)

# 1) If we have a TEAM_ID, test it on a safe problem:
valid_id = False
if TEAM_ID:
    resp = try_select(TEAM_ID, "probatio")
    print("probatio select:", resp.status_code, resp.text)
    valid_id = resp.ok

# 2) If there’s no ID or it’s invalid, register a new team (adding a suffix if needed)
if not valid_id:
    for attempt in range(3):
        suffix = "" if attempt == 0 else "-" + ''.join(random.choice(string.ascii_lowercase+string.digits) for _ in range(3))
        reg = requests.post(
            f"{BASE}/register",
            json={"name": TEAM_NAME + suffix, "pl": LANGUAGE, "email": EMAIL},
            timeout=30
        )
        print("REGISTER:", reg.status_code, reg.text)
        if reg.ok:
            TEAM_ID = reg.json()["id"]
            break
    else:
        raise RuntimeError("Unable to register a team after several attempts.")
    # Verify by selecting probatio
    resp = try_select(TEAM_ID, "probatio")
    resp.raise_for_status()

print("Using TEAM_ID:", TEAM_ID)

# 3) Now select the six‑room problem
sel = try_select(TEAM_ID, "primus")
print("primus select:", sel.status_code, sel.text)
sel.raise_for_status()


probatio select: 400 {"error":"Error: couldn't find matching record"}
REGISTER: 200 {"id":"tarstars@gmail.com X87FWxcYFUh0pd0m2aopZQ"}
Using TEAM_ID: tarstars@gmail.com X87FWxcYFUh0pd0m2aopZQ
primus select: 200 {"problemName":"primus"}


In [ ]:
# Lean learner for primus (6 rooms)
import json, pathlib, requests

BASE = "https://31pwr5t6ij.execute-api.eu-west-2.amazonaws.com"
alphabet = "012345"
results = {}
def explore(plans, chunk=200):
    todo = [p for p in plans if p not in results]
    for i in range(0, len(todo), chunk):
        batch = todo[i:i+chunk]
        r = requests.post(f"{BASE}/explore", json={"id": TEAM_ID, "plans": batch}, timeout=120)
        r.raise_for_status()
        data = r.json()
        for p, seq in zip(batch, data["results"]):
            results[p] = seq
        print(f"Fetched {len(batch)} plans; queryCount:", data["queryCount"])

def label_at(plan, pos): return results[plan][pos]
E = [""]
def row_of(prefix): return (label_at(prefix, len(prefix)),)
reps = [""]
rows = {}
explore([""])
rows[row_of("")] = ""
TARGET_STATES = 6

while True:
    succs = [s + a for s in reps for a in alphabet]
    explore(succs)
    new_reps = []
    for s in reps:
        for a in alphabet:
            sp = s + a
            r = row_of(sp)
            if r not in rows:
                rows[r] = sp
                new_reps.append(sp)
    if not new_reps or len(rows) >= TARGET_STATES:
        break
    reps.extend(new_reps)

print("States discovered:", len(rows), "|E|=", len(E))
reps = [rows[r] for r in rows]
reps.sort(key=len)
idx_of_row = {row_of(rep): i for i, rep in enumerate(reps)}
need = [rep + a for rep in reps for a in alphabet]
explore(need)
n = len(reps)
delta = [[None]*6 for _ in range(n)]
for i, rep in enumerate(reps):
    for a in range(6):
        sp = rep + str(a)
        delta[i][a] = idx_of_row[row_of(sp)]

# build rooms / startingRoom / connections…
# (same as earlier)


Fetched 1 plans; queryCount: 2
Fetched 6 plans; queryCount: 9
Fetched 12 plans; queryCount: 22
Fetched 6 plans; queryCount: 29
States discovered: 4 |E|= 1


In [ ]:
# Lean primus learner (batched, no files)
import requests, json

BASE = "https://31pwr5t6ij.execute-api.eu-west-2.amazonaws.com"
TEAM_ID = "tarstars@gmail.com X87FWxcYFUh0pd0m2aopZQ"  # use EXACTLY the id returned by /register
alphabet = "012345"

results = {}
def explore(plans, chunk=200):
    # batch missing plans
    todo = [p for p in plans if p not in results]
    for i in range(0, len(todo), chunk):
        batch = todo[i:i+chunk]
        r = requests.post(f"{BASE}/explore", json={"id": TEAM_ID, "plans": batch}, timeout=120)
        r.raise_for_status()
        data = r.json()
        for p, seq in zip(batch, data["results"]):
            results[p] = seq
        print(f"Fetched {len(batch)} plans; queryCount:", data["queryCount"])

def label_at(plan, pos): return results[plan][pos]
def row_of(prefix): return (label_at(prefix, len(prefix)),)

# representatives and rows
reps = [""]        # start with empty prefix
rows = {}          # row(tuple) -> representative prefix

# seed
explore([""])
rows[row_of("")] = ""

TARGET_STATES = 6  # primus has 6 rooms

# discover states by only expanding reps one step
while True:
    succs = [s + a for s in reps for a in alphabet]
    explore(succs)
    new = []
    for s in reps:
        for a in alphabet:
            sp = s + a
            r = row_of(sp)
            if r not in rows:
                rows[r] = sp
                new.append(sp)
    if not new or len(rows) >= TARGET_STATES:
        break
    reps.extend(new)

print("States discovered:", len(rows))
reps = [rows[r] for r in rows]
reps.sort(key=len)

# index by row, ensure we have rep+a measured
idx_of_row = {row_of(rep): i for i, rep in enumerate(reps)}
need = [rep + a for rep in reps for a in alphabet]
explore(need)

# transitions
n = len(reps)
delta = [[None]*6 for _ in range(n)]
for i, rep in enumerate(reps):
    for a in range(6):
        sp = rep + str(a)
        delta[i][a] = idx_of_row[row_of(sp)]

# build connections with partner doors
def partner(i,a):
    j = delta[i][a]
    for b in range(6):
        if delta[j][b] == i:
            return j,b
    return None

rooms = [row_of(rep)[0] for rep in reps]
start_idx = idx_of_row[row_of("")]
connections, seen = [], set()
for i in range(n):
    for a in range(6):
        pb = partner(i,a)
        if pb is None: continue
        j,b = pb
        key = tuple(sorted([(i,a),(j,b)]))
        if key not in seen:
            seen.add(key)
            connections.append({"from":{"room":i,"door":a},"to":{"room":j,"door":b}})

map_obj = {"rooms": rooms, "startingRoom": start_idx, "connections": connections}
print("\n=== Candidate map ===")
print(json.dumps(map_obj, indent=2))

# Uncomment to submit:
rr = requests.post(f"{BASE}/guess", json={"id": TEAM_ID, "map": map_obj}, timeout=60)
print("Guess:", rr.status_code, rr.text)


Fetched 1 plans; queryCount: 31
Fetched 6 plans; queryCount: 38
Fetched 12 plans; queryCount: 51
Fetched 6 plans; queryCount: 58
States discovered: 4

=== Candidate map ===
{
  "rooms": [
    0,
    2,
    1,
    3
  ],
  "startingRoom": 0,
  "connections": [
    {
      "from": {
        "room": 0,
        "door": 0
      },
      "to": {
        "room": 1,
        "door": 0
      }
    },
    {
      "from": {
        "room": 0,
        "door": 1
      },
      "to": {
        "room": 0,
        "door": 1
      }
    },
    {
      "from": {
        "room": 0,
        "door": 2
      },
      "to": {
        "room": 2,
        "door": 1
      }
    },
    {
      "from": {
        "room": 0,
        "door": 3
      },
      "to": {
        "room": 2,
        "door": 1
      }
    },
    {
      "from": {
        "room": 0,
        "door": 4
      },
      "to": {
        "room": 0,
        "door": 1
      }
    },
    {
      "from": {
        "room": 0,
        "door": 5
      },
  

In [ ]:
# --- Refining primus: add one distinguishing suffix if needed, rebuild, submit ---
import requests, json

BASE = "https://31pwr5t6ij.execute-api.eu-west-2.amazonaws.com"
TEAM_ID = "tarstars@gmail.com X87FWxcYFUh0pd0m2aopZQ"  # use exactly your returned id
alphabet = "012345"

results = {}
def explore(plans, chunk=200):
    # Validate
    validate_plans(plans)
    # Send only missing plans, in batches
    todo = [p for p in plans if p not in results]
    for i in range(0, len(todo), chunk):
        batch = todo[i:i+chunk]
        r = requests.post(f"{BASE}/explore", json={"id": TEAM_ID, "plans": batch}, timeout=120)
        if not r.ok:
            print("EXPLORE batch failed:", r.status_code, r.text[:400])
            r.raise_for_status()
        data = r.json()
        for p, seq in zip(batch, data["results"]):
            results[p] = seq
        print(f"Fetched {len(batch)} plans; queryCount:", data.get("queryCount"))



def lab(plan, pos): return results[plan][pos]

def row_of(prefix, E):
    tup = []
    for e in E:
        pe = prefix + e
        explore([pe])
        tup.append(lab(pe, len(prefix)))
    return tuple(tup)

def learn_reps(E, target_states=6):
    reps, rows = [""], {}
    explore([""])
    rows[row_of("", E)] = ""
    while True:
        succs = [s+a for s in reps for a in alphabet]
        explore(succs)
        new = []
        for s in reps:
            for a in alphabet:
                sp = s+a
                r = row_of(sp, E)
                if r not in rows:
                    rows[r] = sp
                    new.append(sp)
        if not new or len(rows) >= target_states:
            break
        reps.extend(new)
    reps = [rows[r] for r in rows]
    reps.sort(key=len)
    return reps, rows

def build_delta(reps, E):
    idx = {row_of(rep, E): i for i,rep in enumerate(reps)}
    need = [rep+a for rep in reps for a in alphabet]
    explore(need)
    delta = [[None]*6 for _ in range(len(reps))]
    for i, rep in enumerate(reps):
        for a in range(6):
            sp = rep + str(a)
            delta[i][a] = idx[row_of(sp, E)]
    return delta

def partner(delta, i, a):
    j = delta[i][a]
    for b in range(6):
        if delta[j][b] == i:
            return j,b
    return None

# 1) start with E=[""]
E = [""]
reps, rows_map = learn_reps(E, target_states=6)
delta = build_delta(reps, E)

# 2) add one distinguishing suffix if needed
missing = [(i,a) for i in range(len(reps)) for a in range(6) if partner(delta, i, a) is None]
if missing:
    row_to_ids = {}
    for i, rep in enumerate(reps):
        row_to_ids.setdefault(row_of(rep, E), []).append(i)
    pairs = [(g[x], g[y]) for g in row_to_ids.values() if len(g)>1 for x in range(len(g)) for y in range(x+1,len(g))]
    chosen = None
    for e in alphabet:
        explore([reps[i]+e for i in range(len(reps))])
        diffs = any(row_of(reps[i]+e, [""]) != row_of(reps[j]+e, [""]) for (i,j) in pairs)
        if diffs:
            chosen = e
            break
    if chosen is not None:
        E.append(chosen)
        reps, rows_map = learn_reps(E, target_states=6)
        delta = build_delta(reps, E)

# 3) build and submit map
e0 = 0
rooms = [row_of(rep, E)[e0] for rep in reps]
idx = {row_of(rep, E): i for i,rep in enumerate(reps)}
start_idx = idx[row_of("", E)]
connections, seen = [], set()
for i in range(len(reps)):
    for a in range(6):
        p = partner(delta, i, a)
        if p is None: continue
        j,b = p
        key = tuple(sorted([(i,a),(j,b)]))
        if key not in seen:
            seen.add(key)
            connections.append({"from":{"room":i,"door":a},"to":{"room":j,"door":b}})
map_obj = {"rooms": rooms, "startingRoom": start_idx, "connections": connections}
print(json.dumps(map_obj, indent=2))
rr = requests.post(f"{BASE}/guess", json={"id": TEAM_ID, "map": map_obj}, timeout=60)
print("Guess:", rr.status_code, rr.text)

Fetched 1 plans; queryCount: 10
Fetched 6 plans; queryCount: 17
Fetched 18 plans; queryCount: 36
{
  "rooms": [
    0,
    2,
    3,
    1
  ],
  "startingRoom": 0,
  "connections": [
    {
      "from": {
        "room": 0,
        "door": 0
      },
      "to": {
        "room": 1,
        "door": 1
      }
    },
    {
      "from": {
        "room": 0,
        "door": 1
      },
      "to": {
        "room": 2,
        "door": 1
      }
    },
    {
      "from": {
        "room": 0,
        "door": 2
      },
      "to": {
        "room": 1,
        "door": 1
      }
    },
    {
      "from": {
        "room": 0,
        "door": 3
      },
      "to": {
        "room": 2,
        "door": 1
      }
    },
    {
      "from": {
        "room": 0,
        "door": 4
      },
      "to": {
        "room": 0,
        "door": 4
      }
    },
    {
      "from": {
        "room": 0,
        "door": 5
      },
      "to": {
        "room": 3,
        "door": 2
      }
    },
    {
      

In [ ]:
import requests, re

BASE = "https://31pwr5t6ij.execute-api.eu-west-2.amazonaws.com"
TEAM_ID = "tarstars@gmail.com X87FWxcYFUh0pd0m2aopZQ"  # <-- use EXACT id you got from /register
PROBLEM = "primus"                                    # or "probatio" etc.

def must_ok(r, label):
    print(f"{label}: {r.status_code} {r.text[:200]}")
    r.raise_for_status()

# 0) Re-select the problem to be safe (clears any stale state)
sel = requests.post(f"{BASE}/select", json={"id": TEAM_ID, "problemName": PROBLEM}, timeout=30)
must_ok(sel, "SELECT")

# 1) Strict plan validator (digits 0-5 only; allow "" start)
digit_re = re.compile(r"^[0-5]*$")
def validate_plans(plans):
    assert isinstance(plans, list) and plans, "plans must be a non-empty list"
    for p in plans:
        assert isinstance(p, str), "each plan must be a string"
        assert digit_re.match(p), f"bad plan '{p}': only digits 0-5 allowed"
        # (optional) cap length if you like; API usually allows longish routes
        # assert len(p) <= 2000, "plan too long"

# 2) Safe explore wrapper that prints server errors
def explore_safe(plans):
    validate_plans(plans)
    try:
        r = requests.post(f"{BASE}/explore", json={"id": TEAM_ID, "plans": plans}, timeout=120)
        must_ok(r, "EXPLORE")
        data = r.json()
        print("queryCount:", data.get("queryCount"))
        return data["results"]
    except requests.HTTPError as e:
        # Print full server response to see the real reason for 400
        print("EXPLORE failed. Body:", getattr(e.response, "text", "(no body)"))
        raise

# 3) Tiny probe to confirm everything works before your learner runs
probe = ["", "0", "1", "2", "3", "4", "5"]
print("Probing …")
res = explore_safe(probe)
for p, seq in zip(probe, res):
    print(f"{p!r} -> {seq}")

SELECT: 200 {"problemName":"primus"}
Probing …
EXPLORE: 200 {"results":[[0],[0,2],[0,3],[0,2],[0,3],[0,0],[0,1]],"queryCount":8}
queryCount: 8
'' -> [0]
'0' -> [0, 2]
'1' -> [0, 3]
'2' -> [0, 2]
'3' -> [0, 3]
'4' -> [0, 0]
'5' -> [0, 1]


In [ ]:
map_obj

{'rooms': [0, 2, 3, 1],
 'startingRoom': 0,
 'connections': [{'from': {'room': 0, 'door': 0},
   'to': {'room': 1, 'door': 1}},
  {'from': {'room': 0, 'door': 1}, 'to': {'room': 2, 'door': 1}},
  {'from': {'room': 0, 'door': 2}, 'to': {'room': 1, 'door': 1}},
  {'from': {'room': 0, 'door': 3}, 'to': {'room': 2, 'door': 1}},
  {'from': {'room': 0, 'door': 4}, 'to': {'room': 0, 'door': 4}},
  {'from': {'room': 0, 'door': 5}, 'to': {'room': 3, 'door': 2}},
  {'from': {'room': 1, 'door': 0}, 'to': {'room': 1, 'door': 0}},
  {'from': {'room': 1, 'door': 2}, 'to': {'room': 3, 'door': 1}},
  {'from': {'room': 1, 'door': 3}, 'to': {'room': 3, 'door': 1}},
  {'from': {'room': 1, 'door': 4}, 'to': {'room': 1, 'door': 0}},
  {'from': {'room': 1, 'door': 5}, 'to': {'room': 0, 'door': 0}},
  {'from': {'room': 2, 'door': 0}, 'to': {'room': 3, 'door': 3}},
  {'from': {'room': 2, 'door': 2}, 'to': {'room': 0, 'door': 1}},
  {'from': {'room': 2, 'door': 3}, 'to': {'room': 0, 'door': 1}},
  {'from': {'r

In [ ]:
import json, requests

# -------------------------------------------------------------------------
# Configuration – fill in your secret TEAM_ID from /register:
BASE    = "https://31pwr5t6ij.execute-api.eu-west-2.amazonaws.com"
TEAM_ID = "tarstars@gmail.com X87FWxcYFUh0pd0m2aopZQ"
# Select the 'primus' problem; the call resets any previous state.
r = requests.post(f"{BASE}/select", json={"id": TEAM_ID, "problemName": "primus"}, timeout=30)
r.raise_for_status()
print("select:", r.json())

# -------------------------------------------------------------------------
# Helper to batch /explore calls and cache results.  We mask each label to
# its lowest two bits (mod 4) so that states with the same two‑bit label
# are not falsely distinguished.
alphabet = "012345"
results  = {}  # plan → masked label sequence

def explore(plans, chunk=200):
    """Send any unexplored plans to /explore in batches and cache the results."""
    todo = [p for p in plans if p not in results]
    for i in range(0, len(todo), chunk):
        batch = todo[i:i+chunk]
        resp = requests.post(f"{BASE}/explore", json={"id": TEAM_ID, "plans": batch}, timeout=120)
        resp.raise_for_status()
        data = resp.json()
        for p, seq in zip(batch, data["results"]):
            # clip each observed label to the lowest 2 bits
            results[p] = [label & 3 for label in seq]
        print(f"Fetched {len(batch)} plans; queryCount:", data["queryCount"])

def label_at(plan, pos):
    """Return the masked label at the given position for a plan."""
    return results[plan][pos]

# We use a set of distinguishing suffixes E to differentiate rooms that share
# the same immediate label.  For six doors, taking each door once is enough.
E = [""] + list(alphabet)  # ['', '0', '1', '2', '3', '4', '5']

def row_of(prefix):
    """Return the signature of a prefix: the labels after following each suffix in E."""
    return tuple(label_at(prefix + e, len(prefix + e)) for e in E)

# -------------------------------------------------------------------------
# Discover all rooms for the 'primus' instance.
# reps holds representative access strings (prefixes) to each distinct state.
reps = [""]     # start with the empty prefix
rows = {}       # row(tuple) → representative prefix

# measure the empty plan first
explore([""])
rows[row_of("")] = ""

# Expand reps until we see six distinct rows (primus has 6 rooms)
while True:
    # Explore all one‑step extensions of known reps
    succs = [rep + a for rep in reps for a in alphabet]
    explore(succs)
    new_reps = []
    for rep in reps:
        for a in alphabet:
            sp   = rep + a
            r    = row_of(sp)
            if r not in rows:
                rows[r] = sp
                new_reps.append(sp)
    if not new_reps or len(rows) >= 6:
        break
    reps.extend(new_reps)

print("States discovered:", len(rows))

# Sort representatives by length (shortest first)
reps = [rows[r] for r in rows]
reps.sort(key=len)

# Build a mapping from row signature to state index
idx_of_row = {row_of(rep): i for i, rep in enumerate(reps)}

# Ensure all rep+a are explored, then build the transition table
needs = [rep + a for rep in reps for a in alphabet]
explore(needs)

n     = len(reps)
delta = [[None]*6 for _ in range(n)]
for i, rep in enumerate(reps):
    for a in range(6):
        sp = rep + str(a)
        delta[i][a] = idx_of_row[row_of(sp)]

# Helper to find the partner door j,b such that from room i via door a we
# reach room j and come back via door b
def partner(i, a):
    j = delta[i][a]
    for b in range(6):
        if delta[j][b] == i:
            return j, b
    return None

# Build the final map object
rooms      = [row_of(rep)[0] for rep in reps]  # two‑bit labels for each room
start_idx  = idx_of_row[row_of("")]            # index of the starting room
connections = []
seen_edges  = set()
for i in range(n):
    for a in range(6):
        pb = partner(i, a)
        if pb is None:
            continue
        j, b  = pb
        edge  = tuple(sorted([(i, a), (j, b)]))
        if edge not in seen_edges:
            seen_edges.add(edge)
            connections.append({
                "from": {"room": i, "door": a},
                "to":   {"room": j, "door": b}
            })

# Print out the map.  It will have 6 room labels, as required.
map_obj = {"rooms": rooms, "startingRoom": start_idx, "connections": connections}
print(json.dumps(map_obj, indent=2))

select: {'problemName': 'primus'}
Fetched 1 plans; queryCount: 2


KeyError: '0'

In [ ]:
import json, requests

# --- Config (uses your earlier printed ID) ---
BASE    = "https://31pwr5t6ij.execute-api.eu-west-2.amazonaws.com"
TEAM_ID = "tarstars@gmail.com 9QSCzqm4wQp0qkXXQ9nLtQ"  # update if you re-register
alphabet = "012345"

# Select primus
r = requests.post(f"{BASE}/select", json={"id": TEAM_ID, "problemName": "primus"}, timeout=30)
r.raise_for_status()
print("select:", r.json())

# --- Explore cache (mask to 2 bits) ---
results = {}  # plan -> list[int]

def explore(plans, chunk=200):
    todo = [p for p in plans if p not in results]
    for i in range(0, len(todo), chunk):
        batch = todo[i:i+chunk]
        resp = requests.post(f"{BASE}/explore", json={"id": TEAM_ID, "plans": batch}, timeout=120)
        resp.raise_for_status()
        data = resp.json()
        for p, seq in zip(batch, data["results"]):
            results[p] = [v & 3 for v in seq]  # keep only lowest 2 bits
        print(f"Fetched {len(batch)} plans; queryCount:", data["queryCount"])

# Distinguishing suffixes: empty + each single door
E = [""] + list(alphabet)

def ensure_rows(prefixes):
    plans = []
    for p in prefixes:
        for e in E:
            plans.append(p + e)
    explore(plans)

def row_of(prefix):
    # Make sure all prefix+E are measured before reading
    ensure_rows([prefix])
    return tuple(results[prefix + e][len(prefix + e)] for e in E)

# --- Learn primus (6 rooms) ---
reps = [""]   # representative access strings
rows = {}     # row(tuple) -> representative prefix

# seed
ensure_rows([""])
rows[row_of("")] = ""

# discover up to 6 states
TARGET = 6
while True:
    succs = [rep + a for rep in reps for a in alphabet]
    ensure_rows(succs)
    new_reps = []
    for rep in reps:
        for a in alphabet:
            sp = rep + a
            sig = row_of(sp)
            if sig not in rows:
                rows[sig] = sp
                new_reps.append(sp)
    if not new_reps or len(rows) >= TARGET:
        break
    reps.extend(new_reps)

print("States discovered:", len(rows))

# Freeze reps and index
reps = sorted([rows[sig] for sig in rows], key=len)
idx_of = {row_of(rep): i for i, rep in enumerate(reps)}

# Ensure rep+door measured
ensure_rows([rep + a for rep in reps for a in alphabet])

# Build transitions
n = len(reps)
delta = [[None]*6 for _ in range(n)]
for i, rep in enumerate(reps):
    for a in range(6):
        sp = rep + str(a)
        delta[i][a] = idx_of[row_of(sp)]

# Find partner doors and build connections
def partner(i, a):
    j = delta[i][a]
    for b in range(6):
        if delta[j][b] == i:
            return j, b
    return None

rooms = [row_of(rep)[0] for rep in reps]  # six 2-bit labels
start_idx = idx_of[row_of("")]
connections, seen = [], set()
for i in range(n):
    for a in range(6):
        pb = partner(i, a)
        if not pb:
            continue
        j, b = pb
        edge = tuple(sorted([(i, a), (j, b)]))
        if edge in seen:
            continue
        seen.add(edge)
        connections.append({"from": {"room": i, "door": a},
                            "to":   {"room": j, "door": b}})

map_obj = {"rooms": rooms, "startingRoom": start_idx, "connections": connections}
print(json.dumps(map_obj, indent=2))


select: {'problemName': 'primus'}
Fetched 7 plans; queryCount: 8
Fetched 36 plans; queryCount: 45
Fetched 72 plans; queryCount: 118
States discovered: 6
Fetched 108 plans; queryCount: 227
{
  "rooms": [
    0,
    0,
    3,
    1,
    2,
    1
  ],
  "startingRoom": 0,
  "connections": [
    {
      "from": {
        "room": 0,
        "door": 0
      },
      "to": {
        "room": 1,
        "door": 0
      }
    },
    {
      "from": {
        "room": 0,
        "door": 1
      },
      "to": {
        "room": 0,
        "door": 1
      }
    },
    {
      "from": {
        "room": 0,
        "door": 2
      },
      "to": {
        "room": 0,
        "door": 1
      }
    },
    {
      "from": {
        "room": 0,
        "door": 3
      },
      "to": {
        "room": 2,
        "door": 0
      }
    },
    {
      "from": {
        "room": 0,
        "door": 4
      },
      "to": {
        "room": 2,
        "door": 0
      }
    },
    {
      "from": {
        "room": 0,


In [ ]:
import json, requests, itertools

BASE    = "https://31pwr5t6ij.execute-api.eu-west-2.amazonaws.com"
TEAM_ID = "tarstars@gmail.com 9QSCzqm4wQp0qkXXQ9nLtQ"  # keep as is unless you re-register
alphabet = "012345"

# Reselect primus to reset state
r = requests.post(f"{BASE}/select", json={"id": TEAM_ID, "problemName": "primus"}, timeout=30)
r.raise_for_status()
print("select:", r.json())

# ------------ Explore cache (clip to 2 bits) ------------
results = {}  # plan -> [labels]

def explore(plans, chunk=200):
    todo = [p for p in plans if p not in results]
    for i in range(0, len(todo), chunk):
        batch = todo[i:i+chunk]
        resp = requests.post(f"{BASE}/explore", json={"id": TEAM_ID, "plans": batch}, timeout=120)
        resp.raise_for_status()
        data = resp.json()
        for p, seq in zip(batch, data["results"]):
            results[p] = [v & 3 for v in seq]
        print(f"Fetched {len(batch)} plans; queryCount:", data["queryCount"])

def build_E(depth=1):
    if depth == 1:
        return [""] + list(alphabet)
    if depth == 2:
        pairs = ["".join(p) for p in itertools.product(alphabet, repeat=2)]
        return [""] + list(alphabet) + pairs
    raise ValueError("depth must be 1 or 2")

def ensure(prefixes, E):
    plans = []
    for p in prefixes:
        for e in E:
            plans.append(p + e)
    explore(plans)

def row_of(prefix, E):
    return tuple(results[prefix + e][len(prefix + e)] for e in E)

def learn(E, target_states=6):
    reps = [""]      # representative access strings
    rows = {}        # signature -> rep

    ensure([""], E)
    rows[row_of("", E)] = ""

    while True:
        succs = [rep + a for rep in reps for a in alphabet]
        ensure(succs, E)
        new_reps = []
        for rep in reps:
            for a in alphabet:
                sp = rep + a
                sig = row_of(sp, E)
                if sig not in rows:
                    rows[sig] = sp
                    new_reps.append(sp)
        if not new_reps or len(rows) >= target_states:
            break
        reps.extend(new_reps)

    reps = sorted([rows[s] for s in rows], key=len)
    idx_of = {row_of(rep, E): i for i, rep in enumerate(reps)}
    ensure([rep + a for rep in reps for a in alphabet], E)

    n = len(reps)
    delta = [[None]*6 for _ in range(n)]
    for i, rep in enumerate(reps):
        for a in range(6):
            sp = rep + str(a)
            delta[i][a] = idx_of[row_of(sp, E)]

    # partner search
    def partner(i, a):
        j = delta[i][a]
        for b in range(6):
            if delta[j][b] == i:
                return j, b
        return None

    # Build connections and also validate door uniqueness
    rooms = [row_of(rep, E)[0] for rep in reps]
    start_idx = idx_of[row_of("", E)]
    connections, used = [], set()
    ok = True
    for i in range(n):
        for a in range(6):
            pb = partner(i, a)
            if not pb:
                ok = False
                continue
            j, b = pb
            if (i, a) in used or (j, b) in used:
                # door used twice -> invalid under undirected model
                ok = False
            used.add((i, a)); used.add((j, b))
            edge = tuple(sorted([(i, a), (j, b)]))
            if not any(
                (c["from"]["room"], c["from"]["door"], c["to"]["room"], c["to"]["door"]) == (edge[0][0], edge[0][1], edge[1][0], edge[1][1]) or
                (c["from"]["room"], c["from"]["door"], c["to"]["room"], c["to"]["door"]) == (edge[1][0], edge[1][1], edge[0][0], edge[0][1])
                for c in connections
            ):
                connections.append({
                    "from": {"room": edge[0][0], "door": edge[0][1]},
                    "to":   {"room": edge[1][0], "door": edge[1][1]},
                })

    return {
        "rooms": rooms,
        "startingRoom": start_idx,
        "connections": connections,
        "ok": ok,
        "reps": reps,
        "delta": delta
    }

# -------- First pass with 1-step suffixes --------
E1 = build_E(depth=1)
m1 = learn(E1)
print("\nPASS1 ok:", m1["ok"])
print(json.dumps({"rooms": m1["rooms"], "startingRoom": m1["startingRoom"]}, indent=2))

# If door pairing is inconsistent, strengthen signatures with 2-step suffixes and relearn
if not m1["ok"]:
    print("\nRefining with 2-step suffixes…")
    # Re-use cached results; just ask for any missing 2-step plans
    E2 = build_E(depth=2)
    m2 = learn(E2)
    print("PASS2 ok:", m2["ok"])
    print(json.dumps(m2["rooms"], indent=2))
    map_obj = {"rooms": m2["rooms"], "startingRoom": m2["startingRoom"], "connections": m2["connections"]}
else:
    map_obj = {"rooms": m1["rooms"], "startingRoom": m1["startingRoom"], "connections": m1["connections"]}

print("\n=== Candidate map ===")
print(json.dumps(map_obj, indent=2))

# Uncomment to submit:
# rr = requests.post(f"{BASE}/guess", json={"id": TEAM_ID, "map": map_obj}, timeout=60)
# print("guess:", rr.status_code, rr.text)

select: {'problemName': 'primus'}
Fetched 7 plans; queryCount: 8
Fetched 36 plans; queryCount: 45
Fetched 108 plans; queryCount: 154
Fetched 72 plans; queryCount: 227

PASS1 ok: False
{
  "rooms": [
    0,
    2,
    3,
    0,
    1,
    1
  ],
  "startingRoom": 0
}

Refining with 2-step suffixes…
Fetched 108 plans; queryCount: 336
Fetched 200 plans; queryCount: 537
Fetched 200 plans; queryCount: 738
Fetched 176 plans; queryCount: 915
Fetched 200 plans; queryCount: 1116
Fetched 200 plans; queryCount: 1317
Fetched 32 plans; queryCount: 1350
PASS2 ok: False
[
  0,
  2,
  3,
  0,
  1,
  1
]

=== Candidate map ===
{
  "rooms": [
    0,
    2,
    3,
    0,
    1,
    1
  ],
  "startingRoom": 0,
  "connections": [
    {
      "from": {
        "room": 0,
        "door": 0
      },
      "to": {
        "room": 1,
        "door": 4
      }
    },
    {
      "from": {
        "room": 0,
        "door": 1
      },
      "to": {
        "room": 2,
        "door": 1
      }
    },
    {
      "

In [ ]:
import json, requests

BASE    = "https://31pwr5t6ij.execute-api.eu-west-2.amazonaws.com"
TEAM_ID = "tarstars@gmail.com 9QSCzqm4wQp0qkXXQ9nLtQ"  # same ID you used

# If you already have map_obj from the previous cell, keep it.
# Otherwise, paste your printed values here:
# map_obj = {
#   "rooms": [0,2,3,0,1,1],
#   "startingRoom": 0,
#   "connections": [ ... your list ... ]
# }

def validate_map(m):
    ok = True
    issues = []

    rooms = m.get("rooms", [])
    start = m.get("startingRoom", None)
    conns = m.get("connections", [])

    # 1) basic checks
    if len(rooms) != 6:
        ok = False; issues.append(f"rooms length is {len(rooms)}, expected 6")
    if not isinstance(start, int) or not (0 <= start < len(rooms)):
        ok = False; issues.append(f"startingRoom {start} invalid")

    # 2) door usage: each (room,door) must appear exactly once
    used = {}
    for c in conns:
        for side in ("from","to"):
            try:
                r = c[side]["room"]; d = c[side]["door"]
            except Exception:
                ok = False; issues.append(f"bad connection entry: {c}"); continue
            if not (0 <= r < len(rooms)) or not (0 <= d <= 5):
                ok = False; issues.append(f"out of range endpoint {(r,d)}")
            used[(r,d)] = used.get((r,d), 0) + 1

    # each door should be used exactly once
    for r in range(len(rooms)):
        for d in range(6):
            cnt = used.get((r,d), 0)
            if cnt != 1:
                ok = False; issues.append(f"(room {r}, door {d}) used {cnt} times (need 1)")

    # 3) undirected consistency: partner doors must mutually point back
    # Build quick adjacency by (room,door) -> (room,door)
    adj = {}
    for c in conns:
        a = (c["from"]["room"], c["from"]["door"])
        b = (c["to"]["room"],   c["to"]["door"])
        adj[a] = b
        adj[b] = a
    for a, b in adj.items():
        if adj.get(b) != a:
            ok = False; issues.append(f"not mutually paired: {a} <-> {b}")

    return ok, issues

ok, issues = validate_map(map_obj)
print("VALID:", ok)
if not ok:
    print("\n".join(issues))
else:
    rr = requests.post(f"{BASE}/guess", json={"id": TEAM_ID, "map": map_obj}, timeout=60)
    print("guess:", rr.status_code, rr.text)

VALID: False
(room 0, door 1) used 2 times (need 1)
(room 0, door 2) used 3 times (need 1)
(room 1, door 0) used 2 times (need 1)
(room 1, door 1) used 2 times (need 1)
(room 2, door 1) used 2 times (need 1)
(room 2, door 2) used 2 times (need 1)
(room 3, door 2) used 2 times (need 1)
(room 4, door 0) used 2 times (need 1)
(room 4, door 3) used 2 times (need 1)
(room 5, door 3) used 3 times (need 1)


In [ ]:
import json, requests, itertools

BASE    = "https://31pwr5t6ij.execute-api.eu-west-2.amazonaws.com"
TEAM_ID = "tarstars@gmail.com 9QSCzqm4wQp0qkXXQ9nLtQ"  # same ID you used
alphabet = "012345"

# -- reset to primus each run --
r = requests.post(f"{BASE}/select", json={"id": TEAM_ID, "problemName": "primus"}, timeout=30)
r.raise_for_status()
print("select:", r.json())

# ---------- explore cache (clip labels to 2 bits) ----------
results = {}  # plan -> [labels]

def explore(plans, chunk=200):
    todo = [p for p in plans if p not in results]
    for i in range(0, len(todo), chunk):
        batch = todo[i:i+chunk]
        resp  = requests.post(f"{BASE}/explore", json={"id": TEAM_ID, "plans": batch}, timeout=120)
        resp.raise_for_status()
        data  = resp.json()
        for p, seq in zip(batch, data["results"]):
            results[p] = [v & 3 for v in seq]   # keep only lowest 2 bits
        print(f"Fetched {len(batch)} plans; queryCount:", data["queryCount"])

def build_E(depth):
    if depth == 1:
        return [""] + list(alphabet)
    elif depth == 2:
        pairs = ["".join(p) for p in itertools.product(alphabet, repeat=2)]
        return [""] + list(alphabet) + pairs
    elif depth == 3:
        pairs = ["".join(p) for p in itertools.product(alphabet, repeat=2)]
        trips = ["".join(p) for p in itertools.product(alphabet, repeat=3)]
        return [""] + list(alphabet) + pairs + trips
    else:
        raise ValueError("depth must be 1..3")

def ensure(prefixes, E):
    plans = []
    for p in prefixes:
        for e in E:
            plans.append(p + e)
    explore(plans)

def row_of(prefix, E):
    return tuple(results[prefix + e][len(prefix + e)] for e in E)

def learn_with_E(E, target_states=6):
    # Discover states
    reps = [""]      # representative access strings
    rows = {}        # signature -> rep

    ensure([""], E)
    rows[row_of("", E)] = ""

    while True:
        succs = [rep + a for rep in reps for a in alphabet]
        ensure(succs, E)
        new_reps = []
        for rep in reps:
            for a in alphabet:
                sp  = rep + a
                sig = row_of(sp, E)
                if sig not in rows:
                    rows[sig] = sp
                    new_reps.append(sp)
        if not new_reps or len(rows) >= target_states:
            break
        reps.extend(new_reps)

    reps = sorted([rows[s] for s in rows], key=len)
    idx  = {row_of(rep, E): i for i, rep in enumerate(reps)}

    # Ensure all one-step successors measured
    ensure([rep + a for rep in reps for a in alphabet], E)

    # Transitions
    n = len(reps)
    delta = [[None]*6 for _ in range(n)]
    for i, rep in enumerate(reps):
        for a in range(6):
            sp = rep + str(a)
            delta[i][a] = idx[row_of(sp, E)]

    # Partner door finder
    def partner(i, a):
        j = delta[i][a]
        # Unique door b s.t. back to i
        cands = [b for b in range(6) if delta[j][b] == i]
        return (j, cands[0]) if len(cands) == 1 else None

    # Build connections; also validate uniqueness
    rooms = [row_of(rep, E)[0] for rep in reps]
    start = idx[row_of("", E)]
    used  = {}
    conns = []
    ok = True

    for i in range(n):
        for a in range(6):
            pb = partner(i, a)
            if not pb:
                ok = False
                continue
            j, b = pb
            used[(i,a)] = used.get((i,a), 0) + 1
            used[(j,b)] = used.get((j,b), 0) + 1
            # add undirected edge once
            A, B = (i,a), (j,b)
            if A > B: A, B = B, A
            if not any((c["from"]["room"], c["from"]["door"], c["to"]["room"], c["to"]["door"])==(A[0],A[1],B[0],B[1]) or
                       (c["from"]["room"], c["from"]["door"], c["to"]["room"], c["to"]["door"])==(B[0],B[1],A[0],A[1]) for c in conns):
                conns.append({"from":{"room":A[0], "door":A[1]},
                              "to":  {"room":B[0], "door":B[1]}})

    # door usage must be exactly once
    for i in range(n):
        for a in range(6):
            if used.get((i,a), 0) != 1:
                ok = False

    return {"rooms": rooms, "startingRoom": start, "connections": conns, "ok": ok}

# Try depths 1 -> 2 -> 3
best = None
for depth in (1, 2, 3):
    print(f"\n=== Learning with suffix depth {depth} ===")
    E = build_E(depth)
    m = learn_with_E(E)
    print("ok:", m["ok"], "rooms:", m["rooms"])
    best = m
    if m["ok"]:
        break

map_obj = {"rooms": best["rooms"], "startingRoom": best["startingRoom"], "connections": best["connections"]}
print("\n=== Candidate map ===")
print(json.dumps(map_obj, indent=2))

# Optional: local validator (no submit)
def validate_map(m):
    ok = True
    issues = []
    n = len(m["rooms"])
    used = {}
    for c in m["connections"]:
        for side in ("from","to"):
            r = c[side]["room"]; d = c[side]["door"]
            used[(r,d)] = used.get((r,d), 0) + 1
    for r in range(n):
        for d in range(6):
            cnt = used.get((r,d), 0)
            if cnt != 1:
                ok = False; issues.append(f"(room {r}, door {d}) used {cnt} times")
    return ok, issues

ok, issues = validate_map(map_obj)
print("VALID:", ok)
if not ok:
    print("\n".join(issues))

select: {'problemName': 'primus'}

=== Learning with suffix depth 1 ===
Fetched 7 plans; queryCount: 8
Fetched 36 plans; queryCount: 45
Fetched 36 plans; queryCount: 82
Fetched 108 plans; queryCount: 191
Fetched 36 plans; queryCount: 228
ok: False rooms: [0, 1, 0, 2, 1, 3]

=== Learning with suffix depth 2 ===
Fetched 180 plans; queryCount: 409
Fetched 108 plans; queryCount: 518
Fetched 200 plans; queryCount: 719
Fetched 200 plans; queryCount: 920
Fetched 200 plans; queryCount: 1121
Fetched 12 plans; queryCount: 1134
Fetched 200 plans; queryCount: 1335
Fetched 16 plans; queryCount: 1352
ok: False rooms: [0, 1, 0, 2, 1, 3]

=== Learning with suffix depth 3 ===
Fetched 200 plans; queryCount: 1553
Fetched 200 plans; queryCount: 1754
Fetched 200 plans; queryCount: 1955
Fetched 200 plans; queryCount: 2156
Fetched 200 plans; queryCount: 2357
Fetched 80 plans; queryCount: 2438
Fetched 200 plans; queryCount: 2639
Fetched 200 plans; queryCount: 2840
Fetched 200 plans; queryCount: 3041
Fetched 4

In [ ]:
import json, os, requests

BASE    = "https://31pwr5t6ij.execute-api.eu-west-2.amazonaws.com"
TEAM_ID = os.getenv("ICFPC_ID", "YOUR_SECRET_ID_HERE")  # FIX: prefer env var
alphabet = "012345"

# ---- API helpers -----------------------------------------------------------
def api(path, payload, timeout=120):
    r = requests.post(f"{BASE}{path}", json=payload, timeout=timeout)
    r.raise_for_status()
    return r.json()

def select(problem):
    return api("/select", {"id": TEAM_ID, "problemName": problem}, timeout=30)

def explore_plans(plans):
    """Batch /explore calls, dedup, and return (labels, queryCount)."""
    todo = [p for p in plans if p not in results]
    qcount = None
    # keep batches modest to avoid timeouts; server allows batching (spec)
    for i in range(0, len(todo), 200):
        batch = todo[i:i+200]
        resp = api("/explore", {"id": TEAM_ID, "plans": batch})
        qcount = resp["queryCount"]
        for p, seq in zip(batch, resp["results"]):
            results[p] = [val & 3 for val in seq]  # labels are 2-bit, mask for safety
    return qcount

def guess(map_obj):
    return api("/guess", {"id": TEAM_ID, "map": map_obj})

# ---- Observation-table machinery ------------------------------------------
results = {}                # plan -> label sequence (masked to 2 bits)
E = [""] + list(alphabet)   # FIX: distinguishing suffixes = empty + each door

def ensure_row_available(prefix):
    """FIX: make sure all plans needed to compute row_of(prefix) are explored."""
    needed = [prefix + s for s in E]
    explore_plans(needed)

def label_at(plan, pos):
    return results[plan][pos]

def row_of(prefix):
    """Signature across all suffixes in E (labels at end of each plan)."""
    return tuple(label_at(prefix + suffix, len(prefix + suffix)) for suffix in E)

# ---- Discover the automaton (rooms & transitions) -------------------------
def discover():
    # Seed: select problem and explore empty plan
    select("primus")  # this resets previous selection (per spec)
    explore_plans([""])
    ensure_row_available("")  # FIX: ensure all suffix probes for "" exist

    reps = [""]      # representative prefixes for discovered states
    rows = {row_of(""): ""}

    # FIX: grow until closed — no more new rows after exploring successors
    changed = True
    while changed:
        changed = False
        succs = []
        for rep in reps:
            for a in alphabet:
                succs.append(rep + a)
        # We will need row_of(sp) for each succ, so ensure their suffix probes exist
        # in one go to reduce /explore penalties (spec encourages batching).
        bulk_needed = []
        for sp in succs:
            bulk_needed.extend(sp + s for s in E)
        explore_plans(bulk_needed)

        for rep in list(reps):
            for a in alphabet:
                sp = rep + a
                r  = row_of(sp)
                if r not in rows:
                    rows[r] = sp
                    reps.append(sp)
                    changed = True

    # Build row->index map
    reps_by_len = sorted((rows[r] for r in rows), key=len)
    idx_of_row  = {row_of(rep): i for i, rep in enumerate(reps_by_len)}

    # Ensure successors are explored (should already be, but cheap to assert)
    explore_plans([rep + a for rep in reps_by_len for a in alphabet])

    # Build transition table
    n = len(reps_by_len)
    delta = [[None]*6 for _ in range(n)]
    for i, rep in enumerate(reps_by_len):
        for a in range(6):
            sp = rep + str(a)
            delta[i][a] = idx_of_row[row_of(sp)]

    # Room labels = label for empty suffix for each representative
    rooms = [row_of(rep)[0] for rep in reps_by_len]
    start_idx = idx_of_row[row_of("")]

    # Undirected connections via “mutual partner” pairing
    connections, seen = [], set()
    for i in range(n):
        for a in range(6):
            j = delta[i][a]
            # find b such that going j --b--> i (mutual door)
            for b in range(6):
                if delta[j][b] == i:
                    edge = tuple(sorted([(i, a), (j, b)]))
                    if edge not in seen:
                        seen.add(edge)
                        connections.append({
                            "from": {"room": i, "door": a},
                            "to":   {"room": j, "door": b}
                        })
                    break

    return {"rooms": rooms, "startingRoom": start_idx, "connections": connections}

if __name__ == "__main__":
    # Build the map
    map_obj = discover()
    print(json.dumps(map_obj, indent=2))

    # Optional: submit the guess (uncomment to actually grade & clear selection)
    # NOTE: You should only call /guess once you’re satisfied; it clears the problem.
    # resp = guess(map_obj)
    # print("Correct?", resp["correct"])

HTTPError: 400 Client Error: Bad Request for url: https://31pwr5t6ij.execute-api.eu-west-2.amazonaws.com/select

In [ ]:
# === PRIMUS data dumper & analyzer (Colab-ready) ============================
import json, collections, requests, math, os

BASE     = "https://31pwr5t6ij.execute-api.eu-west-2.amazonaws.com"
TEAM_ID  = "tarstars@gmail.com 9QSCzqm4wQp0qkXXQ9nLtQ"  # keep inline for tablet use
TEAM_ID  = TEAM_ID.split()[-1]  # only the token part if user pasted "email token"
ALPHA    = "012345"
E        = [""] + list(ALPHA)   # distinguishing suffix set we’ll use
CHUNK    = 200

# ---------- tiny API wrapper with readable errors ----------
def api(path, payload, timeout=120):
    r = requests.post(f"{BASE}{path}", json=payload, timeout=timeout)
    if not r.ok:
        raise RuntimeError(f"{path} {r.status_code}: {r.text}")
    return r.json()

def select(problem):  return api("/select",  {"id": TEAM_ID, "problemName": problem}, timeout=30)
def explore(plans):   return api("/explore", {"id": TEAM_ID, "plans": plans})

# ---------- cache of observations: plan -> list[int] ----------
results = {}  # e.g. "01203" -> [l0,l1,l2,l3,l4]

def explore_plans(plans):
    """Batch unexplored plans and cache label sequences (masked to 2 bits)."""
    todo = [p for p in plans if p not in results]
    total_new, last_qc = 0, None
    for i in range(0, len(todo), CHUNK):
        batch = todo[i:i+CHUNK]
        resp = explore(batch)
        last_qc = resp.get("queryCount")
        for p, seq in zip(batch, resp["results"]):
            results[p] = [v & 3 for v in seq]   # 2-bit labels
        total_new += len(batch)
        print(f"  fetched {len(batch)}   (queryCount={last_qc})")
    return total_new, last_qc

def ensure_row_available(prefix):
    """Ensure we can compute row_of(prefix) by probing prefix+each suffix in E."""
    need = [prefix + s for s in E]
    explore_plans(need)

def label_at(plan, pos):
    return results[plan][pos]

def row_of(prefix):
    """Row signature across E: labels observed at end of each (prefix+suffix)."""
    return tuple(label_at(prefix + s, len(prefix + s)) for s in E)

# ---------- crawl: BFS to a given depth just to COLLECT data ----------
def bfs_collect(depth=7):
    """
    Explore all plans up to given depth (inclusive), and for each visited prefix
    also explore (prefix + each suffix in E) so rows are computable.
    """
    q = collections.deque([""])
    seen = set([""])
    level = { "": 0 }

    # seed empty plan + its suffix probes
    explore_plans([""])
    ensure_row_available("")

    while q:
        p = q.popleft()
        d = level[p]
        if d >= depth:
            continue
        # expand successors
        succs = [p + a for a in ALPHA]
        # explore their raw sequences first (cheap to batch)
        explore_plans(succs)
        # ensure rows for successors
        bulk = []
        for sp in succs:
            bulk += [sp + s for s in E]
        explore_plans(bulk)
        # enqueue
        for sp in succs:
            if sp not in seen:
                seen.add(sp)
                level[sp] = d + 1
                q.append(sp)
    return seen

# ---------- analysis helpers ----------
def analyze_rows():
    # Compute rows for all prefixes that are fully available
    rows, reps = {}, {}
    count_skipped = 0
    for p in results.keys():
        # we only treat p as a "prefix" if all p+E are known
        if all(p + s in results for s in E):
            r = row_of(p)
            rows.setdefault(r, []).append(p)
        else:
            count_skipped += 1

    # representatives: shortest prefix per row
    rep_list = []
    for r, plist in rows.items():
        rep = min(plist, key=len)
        reps[r] = rep
        rep_list.append(rep)

    rep_list.sort(key=len)
    print(f"Unique rows discovered: {len(rep_list)}")
    print("Shortest representatives (first 15):")
    for i, rep in enumerate(rep_list[:15]):
        print(f"  [{i:02d}] len={len(rep)} rep='{rep}' row={row_of(rep)}")
    if len(rep_list) > 15:
        print(f"  … (+{len(rep_list)-15} more)")
    return reps, rep_list

def build_partial_delta(reps_sorted):
    """Build delta from representatives using row signatures (may be partial)."""
    idx_of_row = {row_of(rep): i for i, rep in enumerate(reps_sorted)}
    n = len(reps_sorted)
    delta = [[None]*6 for _ in range(n)]
    for i, rep in enumerate(reps_sorted):
        for a in range(6):
            sp = rep + str(a)
            # if row_of(sp) available, fill transition
            if all(sp + s in results for s in E):
                delta[i][a] = idx_of_row[row_of(sp)]
    return delta

def partner_edges(delta):
    """Collect mutual edges and list inconsistencies (non-mutual or missing)."""
    n = len(delta)
    edges, non_mutual, missing = set(), [], []
    for i in range(n):
        for a in range(6):
            j = delta[i][a]
            if j is None:
                missing.append((i, a))
                continue
            # find a back-door b with delta[j][b] == i
            bj = None
            for b in range(6):
                if delta[j][b] == i:
                    bj = b; break
            if bj is None:
                non_mutual.append((i, a, j))
            else:
                edge = tuple(sorted([(i, a), (j, bj)]))
                edges.add(edge)
    return edges, non_mutual, missing

# ---------- run the collection + analysis on PRIMUS ----------
print("Selecting primus …")
print(select("primus"))

print("\nCollecting exploration traces (BFS) …")
seen = bfs_collect(depth=8)        # adjust depth if you want more/less data
print(f"Collected prefixes (<= depth): {len(seen)}")
print(f"Total plans in cache: {len(results)}")

# dump raw observations to a file you can re-load later
dump_path = "/content/primus_results.json"
with open(dump_path, "w") as f:
    json.dump(results, f)
print(f"Saved raw results to {dump_path}")

# quick aggregate stats
max_len = max((len(p) for p in results), default=0)
lens = collections.Counter(len(p) for p in results)
print("\nPlan length histogram (first 15 lengths):")
for L in range(min(lens.keys() or [0]), min(max_len, 15)+1):
    print(f"  len={L:2d}: {lens.get(L,0)} plans")

# analyze rows (candidate rooms)
print("\nAnalyzing rows …")
reps_map, reps_sorted = analyze_rows()

# build partial delta + check consistency
print("\nBuilding partial delta …")
delta = build_partial_delta(reps_sorted)
edges, non_mutual, missing = partner_edges(delta)
print(f"States (unique rows): {len(reps_sorted)}")
print(f"Filled transitions: {sum(1 for i in range(len(delta)) for a in range(6) if delta[i][a] is not None)} / {len(reps_sorted)*6}")
print(f"Unique undirected edges (mutual): {len(edges)}")
print(f"Non-mutual transitions: {len(non_mutual)} (showing up to 10)")
for tup in non_mutual[:10]:
    i,a,j = tup
    print(f"  i={i}, door {a} -> j={j}, but no back-door to i")

print(f"Missing transitions: {len(missing)} (first 20): {missing[:20]}")

# suggest next probes to close gaps (optional)
need_more = []
for i, rep in enumerate(reps_sorted):
    for a in range(6):
        sp = rep + str(a)
        if not all(sp + s in results for s in E):
            need_more += [sp + s for s in E if sp + s not in results]
print(f"\nNext probes to close gaps (unique): {len(set(need_more))}")

Selecting primus …


RuntimeError: /select 400: {"error":"Error: couldn't find matching record"}

In [3]:
import json, pathlib, requests

BASE = "https://31pwr5t6ij.execute-api.eu-west-2.amazonaws.com"
TEAM_ID = "tarstars@gmail.com 9QSCzqm4wPQ0qkXXQ9nLtQ"

r = requests.post(f"{BASE}/select", json={"id": TEAM_ID, "problemName": "primus"}, timeout=30)
print("select:", r.status_code, r.text)
r.raise_for_status()

select: 400 {"error":"Error: couldn't find matching record"}


HTTPError: 400 Client Error: Bad Request for url: https://31pwr5t6ij.execute-api.eu-west-2.amazonaws.com/select

In [4]:
# End-to-end: re-register -> select primus -> collect -> analyze -> save dump
import requests, json, collections

BASE = "https://31pwr5t6ij.execute-api.eu-west-2.amazonaws.com"

# 1) Register to get a fresh id (token)
reg = requests.post(f"{BASE}/register", json={
    "name":  "tarstars",
    "email": "tarstars@gmail.com",
    "pl":    "python"
}, timeout=30)
reg.raise_for_status()
TEAM_ID = reg.json()["id"]
print("New TEAM_ID:", TEAM_ID)

def api(path, payload, timeout=120):
    r = requests.post(f"{BASE}{path}", json=payload, timeout=timeout)
    if not r.ok:
        raise RuntimeError(f"{path} {r.status_code}: {r.text}")
    return r.json()

def select(problem):  return api("/select",  {"id": TEAM_ID, "problemName": problem}, timeout=30)
def explore(plans):   return api("/explore", {"id": TEAM_ID, "plans": plans})

# 2) Select primus
print("\nSelecting primus …")
print(select("primus"))

# 3) Collect data (BFS + suffix probes)
ALPHA = "012345"
E      = [""] + list(ALPHA)
CHUNK  = 200
results = {}

def explore_plans(plans):
    todo = [p for p in plans if p not in results]
    for i in range(0, len(todo), CHUNK):
        batch = todo[i:i+CHUNK]
        resp  = explore(batch)
        for p, seq in zip(batch, resp["results"]):
            results[p] = [v & 3 for v in seq]
        print(f"  fetched {len(batch)} (queryCount={resp.get('queryCount')})")

def ensure_row_available(prefix):
    need = [prefix + s for s in E]
    explore_plans(need)

def row_of(prefix):
    return tuple(results[p][len(p)] for p in (prefix+s for s in E))

# Seed + BFS
print("\nCollecting …")
explore_plans([""]); ensure_row_available("")
q, seen, depth = collections.deque([""]), set([""]), 8
while q:
    p = q.popleft()
    if len(p) >= depth: continue
    succs = [p+a for a in ALPHA]
    explore_plans(succs)
    bulk = []
    for sp in succs: bulk += [sp + s for s in E]
    explore_plans(bulk)
    for sp in succs:
        if sp not in seen:
            seen.add(sp); q.append(sp)

# 4) Analyze rows + partial transitions
rows = {}
for p in list(results):
    if all(p+s in results for s in E):
        rows.setdefault(row_of(p), []).append(p)

reps = [min(pl, key=len) for pl in rows.values()]
reps.sort(key=len)
print(f"\nUnique rows discovered: {len(reps)} (showing up to 15)")
for i, rep in enumerate(reps[:15]):
    print(f"  [{i:02d}] len={len(rep)} rep='{rep}' row={row_of(rep)}")

idx_of = {row_of(rep): i for i, rep in enumerate(reps)}
delta = [[None]*6 for _ in reps]
for i, rep in enumerate(reps):
    for a in range(6):
        sp = rep + str(a)
        if all(sp+s in results for s in E):
            delta[i][a] = idx_of[row_of(sp)]

non_mutual, missing = [], []
for i in range(len(reps)):
    for a in range(6):
        j = delta[i][a]
        if j is None:
            missing.append((i, a))
        else:
            if all(delta[j][b] != i for b in range(6)):
                non_mutual.append((i, a, j))
print(f"\nFilled transitions: {sum(v is not None for row in delta for v in row)} / {len(reps)*6}")
print("Non-mutual:", len(non_mutual), " Missing:", len(missing))

# 5) Save dump for later reuse
dump_path = "/content/primus_results.json"
with open(dump_path, "w") as f:
    json.dump(results, f)
print("Saved raw results to", dump_path)

New TEAM_ID: tarstars@gmail.com yFD_tf-9rd1LJxZsYIBOeg

Selecting primus …
{'problemName': 'primus'}

  fetched 1 (queryCount=2)
  fetched 6 (queryCount=9)
  fetched 36 (queryCount=46)
  fetched 36 (queryCount=83)
  fetched 36 (queryCount=120)
  fetched 36 (queryCount=157)
  fetched 36 (queryCount=194)
  fetched 36 (queryCount=231)
  fetched 36 (queryCount=268)
  fetched 36 (queryCount=305)
  fetched 36 (queryCount=342)
  fetched 36 (queryCount=379)
  fetched 36 (queryCount=416)
  fetched 36 (queryCount=453)
  fetched 36 (queryCount=490)
  fetched 36 (queryCount=527)
  fetched 36 (queryCount=564)
  fetched 36 (queryCount=601)
  fetched 36 (queryCount=638)
  fetched 36 (queryCount=675)
  fetched 36 (queryCount=712)
  fetched 36 (queryCount=749)
  fetched 36 (queryCount=786)
  fetched 36 (queryCount=823)
  fetched 36 (queryCount=860)
  fetched 36 (queryCount=897)
  fetched 36 (queryCount=934)
  fetched 36 (queryCount=971)
  fetched 36 (queryCount=1008)
  fetched 36 (queryCount=1045)
  fe

KeyboardInterrupt: 